# Análise comparativa de modelos

Esse notebook destina-se a uma análise comparativa de diferentes abordagens para predição de sentimento em tweets. O objetivo final é analisar diferentes combinações de modelos, vetorizadores e normalizadores e seus respectivos hiperparâmetros para definir dentro todas as combinações possíveis aquela que tenha uma melhor desempenho geral. Para garantir isso será efetuada uma validação cruzada para cada combinação possível tanto de modelos, quanto de hiperparâmetros.

## Importando dependências

In [1]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
)
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, cross_validate
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import TweetTokenizer
from sklearn import svm

nltk.download("stopwords")
tweet_tokenizer = TweetTokenizer()


[nltk_data] Downloading package stopwords to /home/marvin-
[nltk_data]     linux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importando dados (modificar) 

In [2]:
# df = pd.read_csv(
#     "../data/raw/NoThemeTweets.csv", usecols=["tweet_text", "sentiment"]
# ).sample(3000, random_state=42)


# df = df.assign(
#     number_words=df.tweet_text.apply(lambda x: len(x.split(" "))),
# )  # adiciona coluna com número de palavras

# df.drop_duplicates(["tweet_text"], inplace=True)  # remove textos duplicados
# df.drop(
#     df[df.number_words < 5].index, inplace=True
# )  # remove tweets com menos de 5 palavras

# df["tweet_text"] = df["tweet_text"].apply(
#     lambda tweet: formatar_texto(texto=tweet)
# )  #


In [3]:
# df_raw = pd.read_csv(
#     "../data/raw/NoThemeTweets.csv",
#     usecols=["tweet_text", "sentiment"],
# ).sample(10000, random_state=42)

df_stemming_sem_stopwords = pd.read_csv(
    "../data/processed/df_steamed_no_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
).sample(3000, random_state= 42)

df_stemming_com_stopwords = pd.read_csv(
    "../data/processed/df_steamed_with_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
).sample(3000, random_state= 42)

df_lematizado_sem_stopwords = pd.read_csv(
    "../data/processed/df_lemmetized_no_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
).sample(3000, random_state= 42)

df_lematizado_com_stopwords = pd.read_csv(
    "../data/processed/df_lemmetized_with_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
).sample(3000, random_state= 42)


In [4]:
df_lematizado_sem_stopwords.head(5)


,tweet_text,sentiment
5014,não saber que haw ser sobre o doença de o avri...,0
6817,esperar mesmo e ter sorte de nao ser taxar :(,0
811,quando que eu ir imaginar que dormir ser um pr...,0
3532,meu amiga de o Colégio ta tá com anorexia tamb...,0
4195,ai já sinto sdds de o maratona de harry potter...,0


In [5]:
df_lematizado_com_stopwords.head(5)


,tweet_text,sentiment
5014,saber haw sobre doença avril :(,0
6817,esperar ter sorte nao taxar :(,0
811,ir imaginar dormir problema p eu :(,0
3532,amigo Colégio ta tá anorexiar & gt :( poxa gra...,0
4195,"ai sinto sdds maratona harry potter , acostuma...",0


In [38]:
df_stemming_sem_stopwords.tweet_text


for text in df_stemming_sem_stopwords.tweet_text:
    print(text)




não sab que haw era sobr a doenç da avril :(
esper mesm e ter sort de nao ser tax :(
qu que eu ia imagin que dorm ser um problem p mim :(
minh amig do colégi ta tá com anorex também &gt:( pox ela minh mai thinsp iss me choc demal
ai já sint sdd da maraton de harry potter, voc me acostum mal warn :(
voc nunc sab né meu anj :(
20 ano de miseducation of lauryn hill muit boa matér : )
a daí tenh dó del , coitadinha :( rsrss
o karm não dorm :)
tayn cheg na sal hj tod acab olh p mim e fal “hj eu est só vc tod os dias, acord e vim” ai agr q eu fui perceb q ela me cham de acab :)
aind bem que tal solteiro, tav tri afim de ti, mas tu est namor :p qu diss que est sozinho?
vontad de cusp na car de cert pesso :)
saudad da minh marin :(
:) oi eu quer tenh inter
eu sobr tod as situ possível na vid :)
sem fal no coronel no ce hahahah concord contig que ele tem boa formação, estud em harvard mas concord com a sua últ colocação: sim, ele é sem noç hahahah pass bem, quer :)
quer um livr só do noah eu so

In [7]:
df_lematizado_com_stopwords.head(5)


,tweet_text,sentiment
5014,saber haw sobre doença avril :(,0
6817,esperar ter sorte nao taxar :(,0
811,ir imaginar dormir problema p eu :(,0
3532,amigo Colégio ta tá anorexiar & gt :( poxa gra...,0
4195,"ai sinto sdds maratona harry potter , acostuma...",0


In [8]:
print(
    len(df_stemming_sem_stopwords),
    len(df_stemming_com_stopwords),
    len(df_lematizado_sem_stopwords),
    len(df_lematizado_com_stopwords),
)


3000 3000 3000 3000


In [9]:
import numpy as np

# corpus_raw = df_raw.tweet_text.apply(
#     lambda tweet_text: np.str_(tweet_text)
# )

corpus_stemming_sem_stopwords = df_stemming_sem_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)

corpus_stemming_com_stopwords = df_stemming_com_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)
corpus_lematizado_sem_stopwords = df_lematizado_sem_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)
corpus_lematizado_com_stopwords = df_lematizado_com_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)


# labels_raw = df_raw.sentiment.replace(
#     {"Positivo": 1, "Negativo": 0}
# ).to_list()

labels_stemming_sem_stopwords = df_stemming_sem_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels_stemming_com_stopwords = df_stemming_com_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels__lematizado_sem_stopwords = df_lematizado_sem_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels_lematizado_com_stopwords = df_lematizado_com_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()


In [10]:
print(
    # len(corpus_raw),
    # len(labels_raw),
    len(corpus_stemming_sem_stopwords),
    len(corpus_stemming_com_stopwords),
    len(corpus_lematizado_sem_stopwords),
    len(corpus_lematizado_com_stopwords),
    len(labels_stemming_sem_stopwords),
    len(labels_stemming_com_stopwords),
    len(labels__lematizado_sem_stopwords),
    len(labels_lematizado_com_stopwords),
)


3000 3000 3000 3000 3000 3000 3000 3000


In [11]:
corpus = {
    # "corpus_raw": {
    #     "corpus_data": corpus_raw,
    #     "corpus_labels": labels_raw,
    # },
    "corpus_stemming_sem_stopwords": {
        "corpus_data": corpus_stemming_sem_stopwords,
        "corpus_labels": labels_stemming_sem_stopwords,
    },
    "corpus_stemming_com_stopwords": {
        "corpus_data": corpus_stemming_com_stopwords,
        "corpus_labels": labels_stemming_com_stopwords,
    },
    "corpus_lematizado_sem_stopwords": {
        "corpus_data": corpus_lematizado_sem_stopwords,
        "corpus_labels": labels__lematizado_sem_stopwords,
    },
    "corpus_lematizado_com_stopwords": {
        "corpus_data": corpus_lematizado_com_stopwords,
        "corpus_labels": labels_lematizado_com_stopwords,
    },
}


### Inicialmente será definido um dicionário para cada uma  das etapas na formação de uma abordagem(pipeline), as etapas são:

!["Exemplo de pipeline"](../Diagrama.png)


Cada modelo, vetorizador e normalizador possui seu objeto e um conjunto de hiperparâmetros associados a ele. Para cada abordagem(vetorizador + normalizador + modelo) será feita uma validação cruzada, para garantir a consistência das métricas. Além disso, para garantir uma competição justa, cada abordagem deve ser otimizada com os melhores hiperparâmetros possíveis, para que todas estejam em sua melhor versão. Em vista disso, também é necessário utilizar uma validação cruzada neles.



In [12]:
models = {
    "KNN": {
        "model_obj": KNeighborsClassifier(),
        "hyperparameters": {
            "n_neighbors": [7, 11, 21],
            "weights": ["uniform", "distance"],
        },
    },
    "SMV": {
        "model_obj": svm.SVC(),
        "hyperparameters": {
            "kernel": ["linear", "rbf"],
            "C": [0.1, 0.5, 1, 5, 10],
        },
    },
    "GaussianNB": {
        "model_obj": GaussianNB(),
        "hyperparameters": {
            "var_smoothing": [
                1e-8,
                1e-6,
                1e-4,
                1e-2,
            ]
        },
    },
}

vectorizers = {
    "TfidfVectorizer": {
        "vectorizer_obj": TfidfVectorizer(),
        "hyperparameters": {
            "max_features": [500, 1000, 2000],
            "analyzer": ["word", "char"],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
    "CountVectorizer": {
        "vectorizer_obj": CountVectorizer(),
        "hyperparameters": {
            "max_features": [500, 1000, 2000],
            "analyzer": ["word", "char"],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
}

normalizers = {
    "TrucatedSVD": {
        "normalizer_obj": TruncatedSVD(),
        "hyperparameters": {
            "n_components": [
                10,
                25,
                30,
            ]
        },
    },
}


# Modelos selecionados:  

- ### KNN(k-nearest neighbors):
   O algoritmo KNN é um dos algoritmos clássicos de aprendizado de máquina, usualmente utilizado como algoritmo de classificação a ideia básica proposta  é que pontos semelhantes se encontra próximos um dos outros. Por se tratar de um algoritmo baseado na comparação de dados já existentes, o KNN é considerado um algoritmo do tipo "preguiçoso" já que basicamente decora os pontos do dataset, ou seja, o conhecimento já está diretamente nos dados e não em uma função preditora. No problema em questão sendo uma classificação binaria(positivo ou negativo) a classe definida será a que tiver mais de 50% dos votos. 
  
- Hiperparâmetros:
    - n_neighbors:
      Número de vizinho próximos a ser analisado. A quantidade de pontos é geralmente definida como um número impar para evitar empates na classificação de um novo dado, após isso a classe com maior número de instâncias será a selecionada.
      
    - weights:
      Define se a métrica utilizada será apenas a quantidade, ou se a distância dos pontos terá um peso.

- ### SVC(Support Vector Classification).
  O SVM funciona tentando criar uma hiperplano que separe linearmente os dados em classes diferentes, por exemplo, caso de uma plano 2d é simplesmente uma linha. O critério inicial para  isso é uma hiperplano é que ele consiga separar perfeitamente todos os dados, no caso de haver mais de um hiperplano que faça essa separação é definido como melhor aquele que maximiza a distância das instâncias de cada classe mais próxima. No caso dos dados não sejam linearmente separáveis a priore o SVM consegue aumentar quantidade de dimensões, tornando as classes separáveis dessa forma. No caso da análise de textos, montamos um vetor que represente aquele texto de alguma forma com n-dimensões para montar o hiperplano.

  - Hiperparâmetros:
    - kernel:
    Kernel utilizado para o aumento da dimensionalidade do modelo. Usualmente para aplicações de NLP o linear costuma ser o melhor
      
    - C:
    Parâmetro de regularização, "afrouxa" o critério de separação para ser possível separar mais facilmente os dados.

- ### Gaussian Naive Bayes
  O algoritmo Gausian Naive bayses consiste em fazer uma inferência baseado em várias curvas gaussianas adquiridas através das características do dataset de treino, onde cada uma delas é utilizada como uma parte para definir a probabilidade um dado ser de uma classe específica. No caso de um problema de NLP cada palavra possui sua curva gaussiana, associada com a probabilidade dela ser de uma classe ou outra. Em uma classificação binaria(positiva ou negativa), por exemplo, pode-se partir da pergunta: "Esse texto é positivo?" o algoritmo irá calcular a contagem de cada palavra presente no texto e repassar para cada curva gaussiana respectiva, no final irá tirar um score, a mesma coia será feita para a pergunta: "Esse texto é negativo?", calculando um novo score. Para a pergunta que obtiver o maior score será definida como a classe daquele novo input.
  - Hiperparâmetros:
    - var_smoothing:
    Porção utilizada da maior variância, influencia diretamente na geração da curva.

# Vetorizadores selecionados:  

- ### CountVectorizer
  Essa abordagem faz a contagem das palavras presente para cada uma das instâncias, no caso dessa aplicação tweets, as possibilidades são definidas baseadas no conjunto de todas as palavras possíveis de todos os tweets, o corpus. No final é gerado um vetor com a contagem de palavras presentes em cada tweet.

- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra como features ou palavras.
  - stop_words:
    Define um conjunto de palavras ou não para ser removido dos textos antes da contagem, geralmente é removido palavras que não são relevantes para a análise.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise.
    
    
- ### TfidfVectorizer
  Essa abordagem faz a contagem das palavras por instância(tweets) assim como a CountVectorizer, porém além disso calcula a frequência que essa palavra apareceu baseado em todas as instâncias. Ou seja, uma palavra que aparece muito em um determinado tweet, mas muito pouco nos demais, terá um peso muito maior para a definição da classe daquele tweet. Do contrário, uma palavra que aparece em abundância,  em um tweet, mas é muito comum em todos os outros terá um peso menor.
  
- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra como features ou palavras.
  - stop_words:
    Define um conjunto de palavras ou não para ser removido dos textos antes da contagem, geralmente é removido palavras que não são relevantes para a análise.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise
  

# Normalizador selecionado:

- ### TruncatedSVD()
  Geralmente modelos que trabalham com NPL não lidam bem com vetores com uma grande quantidade de zeros seguidos, devido a numerosa quantidade de palavras possíveis dentro do corpus, as instâncias(tweets) não possuirão a maioria das palavras possíveis no corpus, gerando o problema citado acima. Para contornar isso é necessário reduzir para uma dimensão menor esses dados  que já foram filtrados anteriormente na contagem sendo os mais relevantes. Aplicando o redutor de dimensionalidade SVD, esse vetor espaçado com zeros será reduzido.

- Hiperparâmetros:
  - n_components: Define para quantas features o vetor será reduzido.
  

In [13]:
n_splits_cv = 1
n_splits_gs = 2

all_scores = {}

split_cv = ShuffleSplit(n_splits=n_splits_cv, test_size=0.2, random_state=42)

for corpus_name, corpus_data in corpus.items():

    for model_name, model_data in models.items():

        model_params = {
            f"model__{key}": value
            for key, value in model_data["hyperparameters"].items()
        }

        for vectorizer_name, vectorizer_data in vectorizers.items():

            vectorize_params = {
                f"vectorizer__{key}": value
                for key, value in vectorizer_data["hyperparameters"].items()
            }

            for normalizer_name, normalizer_data in normalizers.items():

                normalizer_params = {
                    f"normalizer__{key}": value
                    for key, value in normalizer_data["hyperparameters"].items()
                }

                # for scaler_name, scaler_data in scalers.items():

                #     scaler_params = {
                #         f"scaler__{key}": value
                #         for key, value in scaler_data["hyperparameters"].items()
                #     }

                param_distributions = {
                    **model_params,
                    **vectorize_params,
                    **normalizer_params,
                    # **scaler_params,
                }

                pipeline = Pipeline(
                    steps=[
                        ("vectorizer", vectorizer_data["vectorizer_obj"]),
                        ("normalizer", normalizer_data["normalizer_obj"]),
                        # ("scaler", scaler_data["scaler_obj"]),
                        ("model", model_data["model_obj"]),
                    ]
                )

                approach_name = (
                    f"{corpus_name}__{model_name}__{vectorizer_name}__{normalizer_name}"
                )

                print(f"Fiting best model to \n{approach_name}", end="\n\n")

                tuned_pipeline = RandomizedSearchCV(
                    pipeline,
                    param_distributions,
                    scoring="f1",
                    cv=n_splits_gs,
                    random_state=42,
                )

                scores = cross_validate(
                    tuned_pipeline,
                    corpus_data["corpus_data"],
                    corpus_data["corpus_labels"],
                    cv=split_cv,
                    scoring=["accuracy", "f1", "recall"],
                )

                all_scores.update(
                    {
                        approach_name: {
                            "scores": scores,
                        }
                    }
                )


Fiting best model to 
corpus_stemming_sem_stopwords__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__GaussianNB__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best

In [14]:
all_scores


{'corpus_stemming_sem_stopwords__KNN__TfidfVectorizer__TrucatedSVD': {'scores': {'fit_time': array([4.45430064]),
   'score_time': array([0.10684037]),
   'test_accuracy': array([0.945]),
   'test_f1': array([0.90544413]),
   'test_recall': array([0.85869565])}},
 'corpus_stemming_sem_stopwords__KNN__CountVectorizer__TrucatedSVD': {'scores': {'fit_time': array([4.13586736]),
   'score_time': array([0.10582232]),
   'test_accuracy': array([0.97666667]),
   'test_f1': array([0.96216216]),
   'test_recall': array([0.9673913])}},
 'corpus_stemming_sem_stopwords__SMV__TfidfVectorizer__TrucatedSVD': {'scores': {'fit_time': array([4.18481255]),
   'score_time': array([0.03712177]),
   'test_accuracy': array([0.995]),
   'test_f1': array([0.99178082]),
   'test_recall': array([0.98369565])}},
 'corpus_stemming_sem_stopwords__SMV__CountVectorizer__TrucatedSVD': {'scores': {'fit_time': array([9.80529404]),
   'score_time': array([0.02002668]),
   'test_accuracy': array([1.]),
   'test_f1': array

In [15]:
all_scores


(
    approach_names,
    fit_times,
    scores_times,
    accuracy_means,
    f1_scores_mean,
    recall_scores_mean,
) = ([], [], [], [], [], [])


for approach_name, score in all_scores.items():
    approach_names.append(approach_name)
    fit_times.append(score["scores"]["fit_time"].mean())
    scores_times.append(score["scores"]["score_time"].mean())
    accuracy_means.append(score["scores"]["test_accuracy"].mean())
    f1_scores_mean.append(score["scores"]["test_f1"].mean())
    recall_scores_mean.append(score["scores"]["test_recall"].mean())

test_data = data = {
    "approach_name": approach_names,
    "fit_time": fit_times,
    "score_time": scores_times,
    "accuracy": accuracy_means,
    "f1": f1_scores_mean,
    "recall": recall_scores_mean,
}


test_data_df = pd.DataFrame(test_data)


test_data_df.style.background_gradient()


,approach_name,fit_time,score_time,accuracy,f1,recall
0,corpus_stemming_sem_stopwords__KNN__TfidfVectorizer__TrucatedSVD,4.454301,0.106840,0.945000,0.905444,0.858696
1,corpus_stemming_sem_stopwords__KNN__CountVectorizer__TrucatedSVD,4.135867,0.105822,0.976667,0.962162,0.967391
2,corpus_stemming_sem_stopwords__SMV__TfidfVectorizer__TrucatedSVD,4.184813,0.037122,0.995000,0.991781,0.983696
3,corpus_stemming_sem_stopwords__SMV__CountVectorizer__TrucatedSVD,9.805294,0.020027,1.000000,1.000000,1.000000
4,corpus_stemming_sem_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD,4.598977,0.109111,0.941667,0.900850,0.864130
5,corpus_stemming_sem_stopwords__GaussianNB__CountVectorizer__TrucatedSVD,4.158924,0.104170,0.873333,0.798942,0.820652
6,corpus_stemming_com_stopwords__KNN__TfidfVectorizer__TrucatedSVD,5.761362,0.138477,0.973333,0.956757,0.961957
7,corpus_stemming_com_stopwords__KNN__CountVectorizer__TrucatedSVD,8.191032,0.124901,0.988333,0.981233,0.994565
8,corpus_stemming_com_stopwords__SMV__TfidfVectorizer__TrucatedSVD,4.459579,0.074709,0.993333,0.989247,1.000000
9,corpus_stemming_com_stopwords__SMV__CountVectorizer__TrucatedSVD,5.218338,0.016094,1.000000,1.000000,1.000000


In [16]:
approach_names = test_data_df.approach_name

import numpy as np


def get_best_model(x):
    if x.name.endswith("time"):
        return approach_names[np.argmin(x.values)]

    return approach_names[np.argmax(x.values)]


In [17]:
from statistics import mode

best_approach_name = mode(test_data_df.apply(get_best_model, axis=0).to_list()).split(
    "__"
)
names = ["corpus", "model", "vectorizer", "normalizer"]

best_approach_dict = {name: value for name, value in zip(names, best_approach_name)}

print(f"Best model is \n{best_approach_dict}")


Best model is 
{'corpus': 'corpus_stemming_sem_stopwords', 'model': 'SMV', 'vectorizer': 'CountVectorizer', 'normalizer': 'TrucatedSVD'}


In [18]:
best_normalizer_hyperparameters = normalizers[best_approach_dict["normalizer"]][
    "hyperparameters"
]
best_vectorizer_hyperparameters = vectorizers[best_approach_dict["vectorizer"]][
    "hyperparameters"
]
best_model_hyperparameters = models[best_approach_dict["model"]]["hyperparameters"]


normalizer_params = {
    f"normalizer__{key}": value
    for key, value in best_normalizer_hyperparameters.items()
}
vectorize_params = {
    f"vectorizer__{key}": value
    for key, value in best_vectorizer_hyperparameters.items()
}
model_params = {
    f"model__{key}": value for key, value in best_model_hyperparameters.items()
}


param_distributions = {
    **model_params,
    **vectorize_params,
    **normalizer_params,
    # **scaler_params,
}


best_obj_vectorizer = vectorizers[best_approach_dict["vectorizer"]]["vectorizer_obj"]

best_obj_normalizer = normalizers[best_approach_dict["normalizer"]]["normalizer_obj"]

best_obj_model = models[best_approach_dict["model"]]["model_obj"]

corpus_final = corpus[best_approach_dict["corpus"]]["corpus_data"]
labels_final = corpus[best_approach_dict["corpus"]]["corpus_labels"]


In [19]:
# normalizer_params = {
#     f"normalizer__{key}": value
#     for key, value in normalizers[best_approach_dict["normalizer"]][
#         "hyperparameters"
#     ].items()
# }


cv_n_splits_final = 5

pipeline = Pipeline(
    steps=[
        (
            "vectorizer",
            best_obj_vectorizer,
        ),
        (
            "normalizer",
            best_obj_normalizer,
        ),
        # ("scaler", scaler_data["scaler_obj"]),
        ("model", best_obj_model),
    ]
)
tuned_pipeline = RandomizedSearchCV(
    pipeline,
    param_distributions,
    scoring="f1",
    cv=cv_n_splits_final,
    random_state=42,
)


scores = cross_validate(
    tuned_pipeline,
    corpus_final,
    labels_final,
    cv=split_cv,
    scoring=["accuracy", "f1", "recall"],
)


In [21]:
print(scores)
print(best_approach_dict)

tuned_pipeline.fit(corpus_final, labels_final)

from joblib import dump
dump(tuned_pipeline.best_estimator_, "../models/tuned_pipelinev2.joblib")


{'fit_time': array([31.52412868]), 'score_time': array([0.01585817]), 'test_accuracy': array([1.]), 'test_f1': array([1.]), 'test_recall': array([1.])}
{'corpus': 'corpus_stemming_sem_stopwords', 'model': 'SMV', 'vectorizer': 'CountVectorizer', 'normalizer': 'TrucatedSVD'}


['../models/tuned_pipelinev2.joblib']

In [42]:
teste = ["Este produto é horrível :)","Eu adoro esse produto :("]

pred_teste = []

for tweet in teste:
    pred_teste.append(stemming((formatar_texto(tweet))))

tuned_pipeline.best_estimator_.predict(pred_teste)

array([0, 0])

In [22]:
from sklearn.metrics import confusion_matrix
y_hat = tuned_pipeline.best_estimator_.predict(corpus_final)

confusion_matrix(labels_final,y_hat)


array([[1999,    3],
       [   4,  994]])

{'corpus': 'corpus_stemming_sem_stopwords', 'model': 'SMV', 'vectorizer': 'CountVectorizer', 'normalizer': 'TrucatedSVD'}


array([1, 1, 1])

In [34]:
import spacy
import re


def lematization(texto: str) -> str:
    nlp = spacy.load("pt_core_news_sm")
    doc = nlp(texto)
    texto = " ".join([token.lemma_ for token in doc])
    return texto


def remover_stop_words(texto: str) -> str:
    stopwords = nltk.corpus.stopwords.words("portuguese")

    texto = " ".join(list(filter(lambda x: x not in stopwords, texto.split())))
    return texto


def formatar_texto(texto: str) -> str:
    texto = (
        re.sub(r"(http\S+)|(@\w+)", "", texto)  # remove links, usuários #
        .replace(".", "")
        .replace(";", "")
        .replace("—", "")
    )

    texto = re.sub(r"(  +)", " ", texto)  # remove espaços duplos
    texto = texto.lower().strip()

    return texto


def stemming(texto: str) -> str:
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    palavras = " ".join([stemmer.stem(palavra) for palavra in texto.split()])
    return palavras


In [35]:
fortaleza = ['Sábado + folga + chuva = maravilha da vida 🥰😍☺️ #fortaleza', 'O pôr do sol de minha cidade. Amo demais 🥰 #Fortaleza #Ceara \nFoto via @Juliana_Mara https://t.co/Nqxdm6oXEq', 'Ariana Grande medalhista do Adauto Bezerra&amp;\nZendaya da escola da prefeitura&amp;\nMichael Cera comendo salgado caro do Ari&amp;\nAnya Taylor Joy tomando chuva na fila do RU.\nCidade da Luz 🥰 #Fortaleza #Ceará https://t.co/5MNwy8z6eK https://t.co/nz0k2dM0lB', 'Dom Pedro Laguna - Aquiraz😀\nhttps://t.co/hz78rqgAHW🏝️\n**\n#aquiraz #fortaleza #ceara #eusebio #caucaia #rebaixados #maracanau #messejana #maranguape #navesinsanas #horizonte #navessocadas #dubstyle #itaitinga #fixa #alturalivre #portodasdunas #fortalezaceara #bhfyp @plosmarcelo✴️ https://t.co/BMA6Qc8xM9', 'Nossa pré-candidatura para Vereadora de Fortaleza tá bonita demais! É o apoio de vocês que nos joga pra frente e chega força do país inteiro.\nOlhaê o nosso querido @MarceloFreixo dando o recado 🥰\n#Psol #Eleições2020 #Fortaleza https://t.co/bNYBe6RAww', 'Seara Praia Hotel-Fortaleza😀\nhttps://t.co/ghkG4nZHHp👍\n*\n#fortaleza #ceara #mp_traveldestination #brasil #saopaulo #fortalezaceara #mp_brazilok #cear #nordeste #mp_brazilok_fortaleza #recife #fortalezace #riodejaneiro #amor #salvador #love #caucaia #bahia #natal #praia #brazil💥 https://t.co/6KOvGzd6Y6', 'Bristol Guararapes Hotel-Fortaleza😀\nhttps://t.co/WDf6XJygpr👍\n**\n#fortaleza #ceara #mp_traveldestination #brasil #saopaulo #fortalezaceara #mp_brazilok #cear #nordeste #mp_brazilok_fortaleza #recife #fortalezace #riodejaneiro #salvador #caucaia #bahia #maracanau #natal #brazil💥 https://t.co/EnsdyaMJbC', 'Mais um #turbulenciaafesta concluído com sucesso!! 🥰 Muito obrigado a todos que estiveram com a gente até de manhã! 😍 Vocês são demais!! 👏🏼👏🏼\n📸 @NaraFassi \n#SeTemXandAviãoEuVou #Turbulência #Fortaleza https://t.co/KqlEVy66us', '#FalaJogador | Fala aí, meu REIMARINHO! 🥰👑\n\n#VamosFortaleza | #Fortaleza | #FortalezaEC https://t.co/iMaUeVLd5N', 'É hoje!!! #CaminhadaRosa em #Fortaleza 🙌🏼🙌🏼🙌🏼\n\n😀Estou muito feliz em ser padrinho este ano desta… https://t.co/cDoWBTdd3i', 'Sendo feliz com os meus copos maravilhosos @anygabrielly @SPlotnikovaBRA @NowUnitedBR @JohnnyRocketsBr @PepsiBr  \nComprei pelo Delivery... corre lá e peça o seu... eu já quero fazer a coleção toda.. são 16 copos 🥰❤️ @JohnnyRockets #iguatemi #fortaleza @Nowun1ted @TeamNowUnited https://t.co/2VLIkbYAs5', 'Vem aí o XIII Evangelizar Fortaleza em casa. Confira o recado do Padre Reginaldo Manzotti. 😀\n➡ Viva esse momento de fé aqui na TV Evangelizar. 📺\n\n#XIIIEvangelizar #EmCasa #Fortaleza #PadreReginaldoManzotti #TVEvangelizar https://t.co/mfpIgd0itk', 'Bom dia, Fortaleza!! Café e tapioca todo dia de manhã. 😀👍 #Fortaleza #Nordeste https://t.co/TLkn4Gw4no', 'Bom dia😀👙☀️💋 #praiadofuturo #fortaleza @ Praia do Futuro https://t.co/VH23ycgwnl', '#tchauRogerioceni e os demais #Vitinho #cesar #Gustavohenrique #Arão #DiegoRibas #René #LéoPereira  alguns mais que esqueci nessa hora de Raiva @Flamengo a e vale lembrar parabéns ao @CearaSC #Fortaleza #Fluminense e demais #Flamengo #Mengão #CRF 🔴⚫🔴⚫ 😠 https://t.co/KEPbALOeJF', '#timecbde | Na #livedasemana relembramos hoje a campanha invicta do Brasil 🇧🇷 no 1º Mundial ISF de Futsal que foi realizado em 2007, na cidade de #Fortaleza (CE). Na ocasião, nossa equipe foi composta por alunos do Colégio Magnum (MG), e por alunas do Colégio Evolutivo (CE). 😀 https://t.co/YcDXMo3U2o', 'Volta feliz d #fortaleza 😀#tempoforte… https://t.co/G02KBWYFHM', 'Gosta de praia, biquíni e mora em #fortaleza ? \nTe convido pra seguir minha loja on-line de Beachwear. Lá no app vizinho 🥰 \n\nhttps://t.co/FTrSD3Q6Qg https://t.co/Obehzyqrrn', '#Fortaleza não decepciona.🥰 https://t.co/Zf1FzTZKQ4', 'Novidade no RioMar Kennedy! Conheça toda a qualidade e sabor da Kopenhagen. Aprecie deliciosos sabores a cada mordida. 🍫☕\n\n📌 Fica localizado no Piso L1, ao lado da Turatti.😀⠀⠀⠀\n⠀⠀⠀⠀\n#RioMarKenendy #NovidadenoKennedy #Novidade #Fortaleza https://t.co/WcQb4i1zl0', 'Sim, eu sou cearense! 😀 #ceara #fortaleza #nordeste #terradosol @ Aterro da Praia de Iracema https://t.co/lDvRGqKbEJ', 'Já pode fazer checkin no Apagão 2015 - Eu Fui!? 😀 #Fortaleza', 'Bom dia meus amores 😎😀👙🌞😍💋 #fortaleza @ Marina Park Hotel https://t.co/C9P2npM5ah', 'Remando remando 😀☀️ rosteffler @anaaouriques #Gutinho #Fortaleza #CE http://t.co/ZNLUfGKsUV', 'Fim de Tarde com essa 🌕 maravilhosa 🥰\n#Fortaleza\n#Brazil https://t.co/04ydno30H7', 'Sensualize com estilo 🥰\n#esperamarido #lingeries #vendasonline #modaintima #vaidarcerto #centrofashion #moda #modaplus #Fortaleza #Ceará #picpay #fazumiti #MaisVoce https://t.co/e8s42vnBle', 'Chega junto! Porque vai ter circo na sua casa! \n\n🎪 Dia 3, 10h da manhá no canal 8.1 @tvcidadefortaleza \n\nSintonize a FM 89.9 #ARádioQueCombinaComVocê 😀\n\n#89FM #CombinaComVc #Fortaleza #Rádio89FM #Circo https://t.co/y3QDz4VtG4', 'Alô @Atletico e @GaloNaVeia no domingo não estarei no @Mineirao pela terceira vez (#FORTALEZA, #GREMIO E #AMERICA) Pois o seu parceiro comercial, @GaloBmg cancelou meu cartão, e não emite um novo, impossibilitando o processo de compra on line! Lamentável 😠😠😠', '#TBT Com a camisa do ídolo!! 🥰❤️ #MarceloBoeck #Fortaleza https://t.co/VCMtTMtQrD', 'A mochila a entrar na Fortaleza de Valença! 🎒 É uma das principais fortificações militares da Europa  e com cerca de 5 km de perímetro amuralhado. 😍 Obrigado pela foto Exmo! ❤️ #Fortaleza #Valença #Minho 😎 #Portugal 🥰 #ViagemàVista 🧳 #ViagemàVistaPeloMundo ✈️ https://t.co/HXSE3mysK1', 'Meu amor e meu xodó na mesma mesa 🥰 #Fluminense  #Fortaleza https://t.co/mpF3KUWo1K', 'Amo esse climinha no meu país! Os dedos dos pés geladinhos 🥰 #Fortaleza https://t.co/yav47somqT', 'Pros fãs de #musicais da #Disney em #Fortaleza :\no #CineSãoLuiz vai exibir #Moana em #3D esse fim de semana !\n #GRATUITO 😀\n@cidade_fortalez https://t.co/XRdkH2KyNN', 'Uma boa pra quem tá afim de conhecer o #rugby aqui em #fortaleza #ceará 😀 https://t.co/eNHcfd9k5U', 'Olha só recadinho da ex dançarinas do @AvioesdoForro_ @XandAviao @SolAlmeidaa 😀\n#fortaleza #aviõesdoforró #xand #solange #brasil #indico https://t.co/VO0Ik59e3j', 'Defina em uma palavra o ano de 2016 pra você? 😀\n\n#MarketingDigital #Ano2016 #brasil #fortaleza', '@tavasconcellos sabe o q mais me diverte pela manhã!? É ouvir vc ri das besteiras do @BoechatBandNews e do @jose_simao. #fã #Fortaleza-Ce 😀', '2016 e tem torcedor q ainda acha que foguete na madrugada ganha jogo. 😠\n#juventude #fortaleza #serieC https://t.co/FUpuYMWlpm', 'Deixando o primogênito no aeroporto para seguir de volta à nova casa, após os feriados conosco. A gente te ama, Vini! 🥰♿️🙌 #travel #Fortaleza #ceara em Aeroporto Internacional de Fortaleza https://t.co/W29UBMKhnU', 'O que tem hoje em #fortaleza pra um casal de férias? 🥰 https://t.co/SxPR1mEAUZ', 'Pelo o menos essa noite vou dormir feliz, obrigado #Fortaleza 🥰🙏', 'De volta 🥰😉 #boy #girls #brasil #hot #bissexual #leke #hot #fortaleza #ce https://t.co/9VmI5CVr3d', '“Moro num país tropical abençoado por Deus e bonito por natureza” 🎼\nE minha cidade é mais linda ainda 🥰\nFortaleza, capital do Ceará, nordeste do meu Brasil 🇧🇷\n.\n.\n.\n.\n#fortaleza #ceara #nordeste #iateclube #sabadou… https://t.co/K6LPH8Tq9R', 'Bom dia! 😀\nPor mais que tudo esteja difícil e que você se sinta sem esperança, não desista! \nDias melhores sempre chegam. \n\nQuem crer digita AMÉM. \n\n#RonaldoMartins #NãoDesista #DiasMelhores \n#Fortaleza #Ceará https://t.co/C0B4oERcLW', 'Um filme com fortes emoções será exibido no #CineRecordEspecial nesta terça-feira (21): Uma Noite de Crime. Filme inédito, na tela da @tvcidadefortaleza 📽\n\nSintonize a FM 89.9 #ARádioQueCombinaComVocê 😀\n\n#89FM #CombinaComVc #Fortaleza #Rádio89FM https://t.co/PknoMuYaBq', 'Hoje é dia daquele que tá sempre pronto para  ouvir, acolher e mediar. Feliz Dia Nacional do Ouvidor! 😀  \n \n#Ronaldo Martins #DiaNacionaldoOuvidor  #Ouvidor #Fortaleza #Ceará https://t.co/WSfHjwab4E', 'Hoje é o dia de quem trabalha para nos fazer sorrir! Por isso, quero parabenizar todos os comediantes, em especial para aqueles que eu já tive o prazer de trabalhar.  😀\n\n#RonaldoMartins #DiadoComediante #Comediante #Bolachinha #Fortaleza #Ceará https://t.co/yYfqCH2Rzn', '#Cearense2018 #FORTALEZA #CEARÁ #FORxCEA       \n\nQUEM É O MAIOR E CAMPEÃO DESSE BRASIL ⚽️😀👊🤞', 'Da série: Por onde andei... 😀 #PraiaDeIracema #Fortaleza #Ceará #AmoViajar #Viagens #Trip… https://t.co/2wrNPcMRNh', 'A cidade que dorme e ilumina a paisagem. Boa noite, Fortaleza, sua linda.🌃👍🏼👏🏼🙌🏼✌🏼😀😍😘🌃#fortaleza… https://t.co/ck97CgHQi1']

Morte = ['O #amor gera a #vida; o #egoísmo produz a #morte. Ninguém pode #serfeliz se não for amado; se não fizer uma experiência de amor. 🥰❤️ Leia o texto e compartilhe! #relacionamento #cancaonova \n\nhttps://t.co/LzFEXOuwMa', 'Vejam o que acontece quando se tenta registrar a #morte de alguém 24h depois da #pfizer ...\n\n😡 Óbvio\nMil e um truques p manter os números baixos... https://t.co/jKDtEyr1Uu', 'O #amor gera a #vida; o #egoísmo produz a #morte. Ninguém pode #serfeliz se não for amado; se não fizer uma experiência de amor. 🥰❤️ Leia o texto e compartilhe! #relacionamento #cancaonova \n\nhttps://t.co/LzFEXOuwMa', 'Um pedacinho De Ontem campeonato ESPORTE QUE EU AMO .. \n#EQUIPEKILLER #Ou #A #MORTE #Oss 👊🏻😤😠🖕🏻 https://t.co/ORn6XK6ptN', 'Um país onde uma pessoa que trama a #morte dos próprios pais, de maneira covarde e cruel, recebe #indulto de #DiaDasMaes.\n\nÉ #imoral. 😡', 'Sério tá feio já 😡realmente tá pagando mico 😳😤😵😱 #morte', 'SE A #PRODUÇÃO DESSA @fazendarecord NÃO COMEÇAR A EXPULSAR. VAI DAR #MORTE. TODOS ESTÃO TENSOS LÁ DENTRO! 💣🔪🔨👊🏽😡😤💩 https://t.co/2D9WkFSXAx', 'Toda tragédia é anunciada nesse buraco de q é esse desgoverno😡\n#BozotrazFomeeMorte\n#Morte\n#DaCultura\n#Ciência\n#Saúde\n☠💀☠💀☠💀☠💀 https://t.co/87clFPNFgc', 'io a cosa pensavo quando ho scelto di fare tedesco?!?! 😡 #morte 🔫', 'vontade de tirar uma sonequinha deliciosa agora mas tenho prova amanha 😠😠😠😠 #morte ao capitalismo', 'E se você que está batendo palma para todo esse circo que está sendo montado, você é cúmplice. Cê entendeu, né?!😠\n*\n*\n*\n#brasil #coronavírus #pandemia #lockdown #quarentena #injustiça #morte #paísesemdesenvolvimento #fome #crianças #mães https://t.co/c4rm91hVP6', 'ai🙄🙄 gente muito bom ✅✅ comer no bandejão 😡😡 da melhor ⭐⭐ do brasil💯💯 #faculdade #estudos #comida #morte #entp_de_magé', 'desmaiei no banho, bati a cabeça no chão e não para de sangrar😀\n#morte', '@SorakaAgressiva @_luiz_hansen_ Sorakinha, vc me respondeu 🥰\n#Morte aos jannistas', 'nossa radfem é hilario JKKKKK 😍🥰 VAMOS ESPALHAR O &gt;&gt;&gt;&gt;FEMINISMO&lt;&lt;&lt;&lt;&lt;&lt;&lt; (femismo) usando a paz e o amor! 😃 #morte ao penis (meme da gretchen) VAMOS SER TRANSFOBICAS E ABOLIR GENEROS UUUUUUUUUUU MORTE AO PENIS KKKKKKKKKKKKKKKKKK HILARIO 😍😍', 'Ordens são Ordens não é Eduardo Pazuello? Mesmo que isso acabe tendo como consequência a #morte de milhares de #pessoa 😡\n\n#forabolsonaro #brasil #politica #academiambl #movimentobrasillivre #eleição2022 #bolsonarogenocida #bolsonaro #ministerio #pandemia #health #saude https://t.co/14DIwcaJjU', '@Estadao @EstadaoPolitica Atitude #DESPREZÍVEL , especialmente diante da #morte de mais de 20.000 #brasileiros 😡', 'só pude pegar o cell pq eta intervalo d aula, meu pai vai pegar ele dnv agr\n#morte 😀🔫', 'Gente até quando vão acontecer mortes por causa de bandido que se infiltra nas torcidas. Isso me deixa arrasada. Tem que mudar essa Lei e torna-la mais dura. 😭 #Briga #Torcida #morte Isso não é torcedor é Marginal! 😡 #CidadeAlerta', '@GreenpeaceBR 👿😈#Ruralistas 😡#Agro ☠️#Agrotóxico 💀☠️#Câncer  \n🌶️🌶️NO ⚫️ DOS OUTROS É 😱😱REFRESCO🎃⚰️☠️#Morte', 'A #MORTE não é  a solução para os  seus #PROBLEMAS!!! \n✌😀✌\n\n#SetembroAmarelo💛\n#DigaNÃOaoSuicídio💛\n#MateEssaIdeia💛\n#ValeAPenaViver💛\n#AlegriaVerdadeira💛', '😀😀😀😀😀 regrann from @aalpino  -  Legado - Cartum para Playboy. Janeiro, 2016. #morte #familia… https://t.co/DlcSD3kWha', 'Qual seria seu último desejo antes de morrer?\nhttps://t.co/S0TBHc0QEn 😀\n#humor #quadrinhos #engraçado #morte https://t.co/T44SerqzL9', '#AGORA #NINGUÉM #VAI #ACREDITAR #NA #MORTE #DOS #URSOS, #MESMO #VENDO #ELES #MORREREM. \n😠😠😠😠😠😠😠😠😠😠😠😠😠😠\n ****** Os seres desumanos já chegaram, em muitos lugares... https://t.co/VHrUpVLfc2', '😠😠😠Não é #AMEAÇA e sim um Aviso Prévio , Mexa com a familia de um homen e tenha uma certeza a que a sua #Morte é... https://t.co/vz2gA2asyt', 'Não queira abraçar ela na minha frente #ciumes #morte 😉😠😬🔫🔪💣', 'Quem inventou horário de verão merece a #morte. 😬😠😬', '@itsmep3dro #morte pedro 🥰❤️', 'Eu &amp; ela, a #morte 😍🥰 https://t.co/IeAQv2FKrL', 'há. Agora ficô Bão 😂🙃🙃🙃😂\nCom isso, o #STF tem duas #Decisões #Conflitantes sobre a #Demanda.\nPor quem irão #Decidir?\nPela #Morte ou pela #Vida\nPelo(a) 🤥🤑😈👿😈🤑🤥 ( #Capiroto )\nou\nPelo(a) 🥰🇧🇷😇😇🇧🇷🥰 ( #Humanidade )\n\nhttps://t.co/ts5DFtohmX', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #telhados #casa #casas #house #pulo #like4likes #likememe #nature #videos #video #instavideo #videoviral #engraçado… https://t.co/ts4bDN6g3B', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #lenhador #arvore #madeira #tree #machado #like4likes #likememe #nature #videos #video #instavideo #videoviral #engraçado… https://t.co/4MqEjfsqO8', 'Mais um #capítulo\nda #série #emorreu\nHoje tem dois heim.\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #adestrador #jacarei #zoo #zoologico #alligator #like4likes #likememe #zoologia #video #videos… https://t.co/2J0xfoB19A', 'Mais um #capítulo\nda #série #emorreu\nHoje tem dois heim.\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #bicicleta #bike #fogo #fire #welcometoibiza #caixao #dança #seriados #videos #video #videogram… https://t.co/WKPVUO9i0P', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #garrafa #bottle #bola #ball #welcometoibiza #caixao #dança #seriados #videos #video #videogram #instavideo #videofun… https://t.co/CqNFmaODe6', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #luz #redeeletrica #blackout #cemig #eletricidade #eletricista #curtocirtuito #welcometoibiza #caixao #dança #seriados… https://t.co/I7pO2lTAPn', 'Mais um #capitulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #mortefeliz #dead #paraquedas #aviao #saltodeparaquedas #caiudeparaquedas #aeronaves #welcometoibiza #caixao #dança #seriados… https://t.co/7I5BtS3cxl', 'Faltou ontem porque todos estavam em #lives de #shows no #youtube mas hoje tem dois capítulos de #emorreu \n*\nOuça a gente vai. Informações na Bio 🎧🥰\n*\n #morte #dead #🔞 #welcometoibiza #caixao #mortefeliz #dança… https://t.co/egX8iXVovL', 'Leitura do dia 🥰 Livro Viver o seu Morrer de Stanley Keleman.\n\n#livros #luto #morte #viver #psi #psicologo #psicologia #psicoterapia #saudemental #bemestar #psicologiaporamor #procureumpsicologo… https://t.co/HIk6p0Q23m', 'NUNCA ESQUEÇA: #lula do #pt APÓIA A russia NESTA #GUERRA E CONDENA A #UCRÂNIA, COMO SE A CULPA POR TODA ESTA #DESTRUIÇÃO E #MORTE DE CIDADÃOS FOSSE AGORA DO PAÍS QUE ESTÁ SENDO DEVASTADO!!\n\nVote sim no #exPresidiário Brasil e verás o que lhe espera!!! 😡 #povoBURRO https://t.co/2YbgDINlz1', '😡 COVARDIA\nA mãe confessou ter notado um sangramento após o homem ficar a sós com a neném.\n\n🔸️ Saiba mais sobre o caso ⤵️\nhttps://t.co/C4ncbJpF2M\n\n#prisão #covardia #santacatarina #estupro #morte #nenem #pcsc', 'Esta é a verdadeira #democracia da #esquerda: #guerra, #medo e #MORTE!! 😡 Prepare-se Brasil: #lula do #pt está voltando! https://t.co/T0WmwnsXek', '3 dias com dor de cabeça da sinusite.\nEitaaaa dorzinha infernal!\n😡😡🤬🤬🤬\n#sinusite #morte #aaaaaaaaa https://t.co/y0UvHfLXtp', 'Sabemos onde está a #mulher que sofreu #assalto na #Supervia, só não sabemos do assaltante 😡\n@PCERJ @PM_RJ @SuperVia_trens #assalto #roubo #morte #violencia @radiotupi  @cbnrio @bandnewsfmrio', '@jurandirsas Saudações Patriotas !🇧🇷\nEu gostaria de poder ser à #Morte. Por um só dia, para poder mandar para as profundezas do infinito, todos esses... \nAntipatriotas, Corruptos, Comunistas, Esquerdistas, Neo Nazistas, #PTistas, #FabianosMarxistas, Terroristas, E criminosos de modo geral.😡 https://t.co/YO71jxmcgZ', 'Em Contagem (MG), 50 cães morrem com suspeita de envenenamento. Animais estavam no Lar Temporário, localizado no Bairro Quintas do Jacuba 😡https://t.co/pGXxZ4b1Su #cãesegatos #abrigo #animais #morte #investigação #envenenamento #denúncia https://t.co/1JuDdcY2ql', '@aureoboy sim! isso é pouco, mi amigo! o sabonete infelizmente deixa rastros e bacterias espalhados em todo meu corpo! um absurdo.... tem que morrer mesmo #morte ao sabonete #peste 😡', '@Pontifex_it @Avvenire_Nei @oss_romano @iamCARITAS #Voce grossa contro chi #produce #morte!!!😡 https://t.co/nS6KWURiNh', '@clagraca #VivaVenezuela ✊😡#VivaLaRevulucion #Morte À @jguaido Puto Desgraçado', '@Calks_Real seu machista homofibico  😘😘 eu sou a vitoria machistona do pedaço adoro homem e vc vai ter que se submeter a mim 😡🤬🤬\n#morte ao penis KKKKKKKK 🥺🤣😂😂😅😆']

Acidente = ['Deu ruim😡😢#plantaoenfoco\nAcidente com ônibus deixa feridos na Alameda\nLeia a matéria: https://t.co/JUdHA8Xrki \n\n#niteroi #transito #acidente #plantaoenfoco https://t.co/MvaAX3hENp', 'Bom dia para o “igenheiro” q coloca a segurança em 1 lugar! 🙄😒😡\n#segurancadotrabalho #transito #estrada #acidente https://t.co/0Q1li8ruhB', 'Enquanto isso... pelo salto Merece uma nota 8... encostou a mão no chão.... 🤣😂😅😆😁😃😀😀😃😄😆😅😂🤣🤣\n\ndiga ai qual nota você daria ?\n\n#Salto #acidente #olimpíadas #saltoornamental em Parauapebas https://t.co/yv73qTTjff', 'Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguro #seguroauto #seguromotocicleta https://t.co/fXUsosMfrE', '🚗Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguro #seguros #seguroauto #automoveis #auto #carro #protecao https://t.co/fsfCcvopSt', 'Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguroviagem #viagem #seguro #planodesaude #planosdesaude https://t.co/PgVSnU9wQz', '#gt #bikegt #bicicleta #bicicletagt #acidente #atropelado \n.\n😶🙃😶😀😐🙃😀🚲💥🚗 \n..\nTa bom.... Foi… https://t.co/3iUWwo0AUp', '😱😱😱\nE ainda fugiu sem prestar socorro!\nInadmissível o tamanho da audácia 😠😡\n\n.\n\n#prontofalei #peoples #PARE #acidentes #duasrodas #atropelamento #perigo #sqn #zoando #pessoa #rindo #não é o que #parece #acidente… https://t.co/iy2zBxqsXF', '#Acidente após entrada da Ilha RT @DanyeleDanyreis @InformeRJO Av. Brasil parada na seletiva sentido zona Oeste pq? 😠', 'Que explicação tão torpe 😡😡😡\n\nEntão a vítima foi a viatura? A vida perdida foi a da viatura? #apanhadosrtp #portugal #apanhados #rtp #eduardocabrita #acidente https://t.co/t2LaCf6sEU', '😱😱😱\nE ainda fugiu sem prestar socorro!\nInadmissível o tamanho da audácia 😠😡\n\n.\n\n#prontofalei #peoples #PARE #acidentes #duasrodas #atropelamento #perigo #sqn #zoando #pessoa #rindo #não é o que #parece #acidente… https://t.co/iy2zBxqsXF', 'Acidente na #rodoviacastelobranco tudo parado sentido interior. 😡#acidente #castelobranco #rodovia #transito', 'Mais é um carai mesmo 😡😡😡 #transito #acidente #fernaodias #mioorolê @ Guarulhos S.p https://t.co/tCbdKflxGL', 'Meu dia hoje começou de primeira! 😩😡🚌🚎🚐🚑🚒🚓🚗🚕🚙🚚🚛 #Engarrafamento #TudoParado #EPTG #Acidente #Taguatinga http://t.co/8kBugXhH86', 'Eu desisto! Toda vez é isso!!!! 😒😒😒😡😡😡😡 #engarrafamento #acidente #estrada #partiuBE #stresse #BR101 @… http://t.co/Ggdy5IW6qc']

Marvel = ['Esto me da vida 🥰 #Marvel #MightyThor #GirlPower https://t.co/m6wmSHGNu7', 'Metadinha #TChalla e #Nebulosa! 😍🥰\n\nQuem aí shippou o casal de #WhatIfMarvel? \n\n.#WhatIf #Marvel https://t.co/CeCST5kOF1', 'BABY YODA DE LOKI PASSANDO EM SUA TELA! 😍\n\nQUANTA FOFURA! 🥰\n\n#BabyYoda #StarWars #Loki #Marvel https://t.co/xsW2kHNcfk', 'QUE FOFURA! 🥰\n\nE esse cosplay de #MoonKnight! Ficou perfeito! 😍🌙\n\n#Marvel #CavaleiroDaLua https://t.co/576pIB1xoS', '“Como você prepara sua massa?” DO JEITO QUE VOCÊ QUISER, @dannyramirez 🥰 A @disneyplusbr ouviu o pedido das migas e trouxe o próprio Neném do Macarrão pra bater um papinho com a gente! Foi um sonho, olha só: https://t.co/3MVdPmTcQS #TFATWS\xa0#Marvel #EntreMigas https://t.co/2XRTofrh11', 'Aproveitar que essa é a última semana de felicidade no lokiverso e postar meu desenho do #kidloki \nRt + fav = 🥰\n\n#arte #desenho #art #Loki #LokiWednesdays #lokialligator #Marvel https://t.co/4GdUA8NY9E', 'Eu tô paralisada com esse treiler de moon knight nova série da Marvel Studio 😮🥰\n\n#MoonKnight #Marvel https://t.co/VxYZkxgUwa', 'QUEREMOS 🥰\n\nKaren Gillan tem interesse em fazer uma série como Nebulosa no Disney Plus.\n\n“Eu acho que ela tem uma história muito interessante, então seria muito legal saber mais sobre." #Marvel #GuardiansOfTheGalaxy https://t.co/p6GaSOtQYb', 'WANDAVISION❤️(GIF)\nQuem já assistiu? Tem sido uma das minhas séries favoritas ultimamente 🥰\n•\n\n#wandavision #wandamaximoff #wanda #wandafanart #marvelbrasil #marvel #disney #disneyplus #seriedisney #procreate #art #arte #illustration #ipadart #digitalart #fanart #desenho l https://t.co/XmxKD7YVyp', 'Quando o me perguntam porque amo tanto a Marvel 🥰😍❤🤜🏻🤛🏻\n#Wiccano #Hulkling #MarvelStudios #Marvel https://t.co/BCEa7EnARQ', 'Meu desenho da Sue Storm-Richards, a Mulher Invisível do Quarteto Fantástico. 👱\u200d♀️4️⃣\n\nJá estava devendo a um bom tempo, se você puderem curtir e da RT e muito importante 🥰\n\nNão deixem flopar 🥺\n#Marvel #Fanarts #FantasticFour #InvisibleWoman #SueStorm #QuartetoFantastico https://t.co/s8qhB4sm1T', 'E a performance cômica do ano vai para:\n\nWanda e Visão 🥰\n\nEsse casal começou com o pé direito! \n\n#WandaVision #Marvel #DisneyPlus https://t.co/biYAp4Bccn', '🐕\u200d🦺 Bienvenue à #Marvel (2 mois), nouvelle recrue de la famille ! 🥰 https://t.co/zzpwuB24SU', 'Namoral amei essa aquarela ❤️\n\n🔄+❤️= Eu felizinho 🥰\n\n#fanart #Deadpool #Marvel https://t.co/Fdo8DYpxG2', 'Qual deles é o melhor HULK na sua opinião? Não esqueça de deixar seu voto nos comentários! 😠😠😤\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\n#hulk #escolha #melhor #maisgeektv #filme #mcu #marvel #cinema https://t.co/wawiTJPsGl', 'HAPPY BIRTHDAY @Fer26Ari ❤❤\n\nHoje nossa administradora está completando 18 anos 🥰\nParabéns Fernanda (Nossa Capitã Marvel), aproveite seu dia! A Legião te ama ❤\n\n#WandaVision #Marvel #CaptainMarvel https://t.co/3nMHAQXd5C', 'deadpool pq sla só deu vontade d fazer nem curto tanto marvel.\n.\n.\n.\n.\n.\n( Plz Fav❤️&amp; RT🔁= i happy 🥰)\n.\n.\n.\n.\n.\n(#Art #drawing #anime #animeArt   #originalcharacters #fanart #doodles #sketch #digitalart #MyArt #ArtistOnTwitter #Deadpool #marvel) https://t.co/P96GOii8jP', 'Coleção atualizada!\n#nerd #HQs\n#DCComics #Marvel #DragonBall #StarWars\n#IronStudios #Eaglemoss #Kotobukiya\n#Bandai\n#Superman #Thor #Shazam #Batman\n#WonderWoman #HalJordan #Thanos #Galactus #Darkseid\n✌✌✌✌✌\n😀😀😀😀😀\n👊👊👊👊👊 https://t.co/orNma7ZknD', 'Rostinho novo para a página ✨❤️\n\nps: tô muito feliz com o resultado desse desenho 🥰\n\n#JeanGrey #XMen #Marvel https://t.co/SBrE6QQtZQ', 'Os Defensores (@TheDefenders) na bela arte de Neal Adams. 😀\n#JessicaJones #IronFist #PunhosdeFerro #LukeCage #DareDevil #Demolidor #Marvel #Netflix #f2fstore https://t.co/PzA6Oy6GYI', 'Vou postar aqui pq o insta ferrou cm bagulho, fiz uma arte do aranha que me orgulhei. Finalmente 🥰\n-\n-\n-\n-\n#art #digitalpainting #draw #drawing #artwork #marvel #spiderman #ilustra https://t.co/Qf2QugjQ2Q', 'Te amo #marvel 🥰 https://t.co/RMgMZnbypb', 'Não priemos cânico!" 😀🙃😛\nAdorei esse Chapolin do amigo @EricBlakeArt ! Tive que colorir!\nCores por mim Marcos Martins @mmartinsart .\n#art #chapolin #chapolincolorado #avengers #infinitywar #vingadores #marvel #televisa #thor #thanos #artist #illustration #sketch #mmartinsart https://t.co/e6qjrGgPmP', 'Um sonho que se realiza.\n\nCosplay do Wiccano/Demiurgo da Marvel. 🥰\n\nAgora é iniciar as gravações do meu projeto, e depois esperar pela CCXP 22 para usar ele pleno.\n\n#BillyKaplan #Wiccan #Marvel #MarvelCosplay #WiccanCosplay #Demiurge #Cosplay #YoungAvengers #JovensVingadores https://t.co/3X18LncioY', 'Já estamos com saudade dessa variante.💚 (E de todas as outras)\n\nSylvie, a gente te ama! 🥰\nSophia Di Martino também te amamos!\n\n#Loki #Marvel #Disney #Sylvie #Mobius https://t.co/LPSuN9oHHY', 'Alguns cadernos à pronta entrega com envio imediato! 🥰❤😍\n#OneDirection \n#StarWars \n#TheMandalorian \n#SHINee \n#Marvel\n#Given https://t.co/XAoLYabGXC', "#JTComics #104 : Quand t'assumes pas tes dessins chez #Marvel, un #comics GRATUIT par le papa de #Deadpool...😀 https://t.co/h9gbIeIohK", "Oii gente, Quinto desenho finalizado :)\nThor, Deus do Trovão\nComissões abertas de desenho\nChama DM pra orçamento hehe\n[Rt's são bem vindos :)🥰]\n #SixFanarts #ilustração #Marvel #Thor #thorloveandthunder https://t.co/R5qM8pso4N", 'reizinhos 🥰\n\n#NewMutants #IllyanaRasputin #Magik #RobertodaCosta #Sunspot #fanart #Marvel #XMen https://t.co/vKjJqziJK6', 'Conhece a Mulher-Hulk? 😀\n\n#Marvel #marvelcomics #MarvelStudios #Disney #DisneyPlus #TV #serie #culture #cultura #news #noticias\n\nhttps://t.co/JYoXR41JZo', 'Opaaa já estamos na Nerd Party.  😀\n#manaus #evento #venham #Geek #marvel #gotg #dccomics #batman #wonderwoman #aquaman #flash #greenlantern #nerd https://t.co/fHetWNxRZ2', '😠\n\nSiga @AlternativaNerd\n\nTAGS\n#alternativanerd #mundonerd #thor #marvel #games #gamers #hqs #mundogamer https://t.co/cJ6G80iURp', 'Mano eu tô super ansioso por esse Thor 4 💥\n\nDepois dos críticos só elogiarem o filme então. 🥰\n\n#ThorLoveAndThunder \n#Marvel', 'Quem disse que  #Marvel e #Broadway não combina? O Hugh Jackman está em cartaz com The Music Man e um fã resolveu fazer graça durante o agradecimento! 🥰\n\nObs: Queria deixar claro q essa adm apoia o musical dos Vigadores mostrado durante a série do Gavião Arqueiro... 👀 https://t.co/Ct1QM47acf', '❤ FELIZ ANIVERSÁRIO Chris Evans❣❣Obrigada por nos presentear com o Capitão América. 💞Toda a felicidade do mundo em sua vida.🥰 #Marvel @szilviade_ @Arcelin31705516 @ChrisEvans https://t.co/iiL1BR6Pid https://t.co/I1hdRHAYIy', 'Grupo: MARVEL.  \n\nCASO TENHA INTERESSE EM ENTRAR MANDO O LINK NA SUA DM. 🥰😘 \nCERTIFIQUE-SE QUE A SUA DM ESTÁ LIBERADA OU ME SIGA PARA MANDAR. 👍🏻\n\nRT PARA DIVULGAR. 🤗 \n \n#marvel #MarvelStudios #MarvelsAvengers #MarvelComics #hq https://t.co/lg9T6xk9i8', 'Bloopers - Blindspot S03xE13\nEu amo erros de gravação e AMO MAIS ainda a Risada da JAIMIE 😍❤🥰\n.\n.\n#jaimiealexander #blindspot #pontocego #janedoe #remibriggs #ladysif #thor #thor4 #sullivanstapleton #audreysparza #ashleyjohnson #ennisesmer #mcu #nbc #marvel #disney https://t.co/R5AzJTYPB0', 'Já que é aniversário da #ScarlettJohansson 🥰 fica aqui nossa singela homenagem e a prova que a #Marvel demorou pra fazer o filme! ➡️ https://t.co/7nmCYelqDr', 'Eaiii galerinha do twitter!?! Já fez sua boa ação de hoje??? Que tal doar um chocolatinho pra uma criança só pra fazer a Páscoa dela mais feliz?? 🥰🐰🍫 chamaaaa 🗣❤️\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#pascoa #pascoa2022 #MarvelStudios #JustinBieber #SpiderMan #DoctorStrange #TomHolland #Marvel https://t.co/PmzfnwOdvL', 'É lindo demais ter uma amiga como a Maria. Ela acertou em cheio no amigo oculto. Obrigado, pequenina! 🥰🙌🏻 #Marvel #SpiderMan https://t.co/mlcyftZcTS', 'A primeira série de animação da #Marvel e #DisneyPlus chega a 11 de agosto! 🥰 #WhatIf\nhttps://t.co/6U2IpONqTJ', 'É pedir de mais pelo menos a animação? 🥰😂\n\n#homemaranha #tobeymaguire #marvel #universomarvel #tomholland https://t.co/UuKr55cMvW', '❣Um lindo dia aos apaixonados ❣#Marvel 🥰 https://t.co/H7LsV2z2hl', 'A escola em que minhas filhas estudam, hoje realizaram a entrega de um kit estude em casa. Com isso o aluno poderia escolher dois livros de presente. Olha o livro que ela escolheu e disse que é presente pra mim. \n\n#BlackPanther #Marvel 🥰\n\nO #SherlockHolmes é dela! https://t.co/zDLuDaV2U5', 'O Wolverine chegou no @fortnite galera! 🙌🙌🙌\nE hoje vamos fazer uma  live ESPECIAL as 22h na @twitch para tentar liberar essa Skin top! 👊😀🎮\nLink https://t.co/Tr84CJuswE\n\n#wolverinefortnite #fortnite #twitch #twitch #angryplaygamers #epicgames #marvel #wolverine https://t.co/sAfe8vn0tg', 'Adorei o filme Homem-Aranha no Aranhaverso! Bem divertido, bem feito, emocionante. \n\nJá tô atrás da trilha sonora 😀\n\n#HomemAranhaNoAranhaverso \n#Marvel', 'O sorriso mais lindo 😀❤✌\n.\n#finnjones #ironfistseason2 #ironfist #punhodeferro #dannyrand #thedefendersnetflix #thedefenders #lukecageseason2 #netflix #marvel #marvelcomics #marvelstudios #marvelironfist #marvelcinematicuniverse #marvelnetflix #heros… https://t.co/DucjVViWHV https://t.co/JCXeJXztqy', 'Capitão América vs Próxima Meia-Noite \nCapitão pegou a referência lançada pela Próxima com sucesso!\nMarvel Libera logo esse trailer pleaseee!\n#AvengersInfinityWar #Avengers4 #Marvel #Avengers ☺️😀🤓🧐📽️💻 https://t.co/iqwFvzUI4N', 'Bom agora que eu vi #Ultimato ou #AndGame nunca mais assisto filmes #Marvel que porra fizeram com a Romanoff e o Rogers?? Mano tava tudo bom ai cagaram tudo pqp. #TomanocuRoteristademerda 😠😠😠😠😠😠😠😠😠😠😠😠😡😡😡😡😡😡😡😡😡😡😡', 'É hoje papai… as 19hrs 🥰\n#Marvel #doutorestranho2 #DoctorStrange2 #MarvelStudios #DrStrangeMultiverseOfMadness #DoctorStrange # https://t.co/5JVmbnzIBm']

In [36]:

print(best_approach_dict)
pred_fortaleza = tuned_pipeline.best_estimator_.predict(fortaleza)

pred_fortaleza_formated = []

for tweet in fortaleza:
    pred_fortaleza_formated.append(stemming((formatar_texto(tweet))))



df_teste = pd.DataFrame(zip(fortaleza, pred_fortaleza), columns=["text", "class"])

df_teste.to_csv("../result_pipeline_test/fortalezav2.csv", index=False)


# for tweet in Morte:
#     teste_morte.append(lematization(remover_stop_words(formatar_texto(tweet))))

df_teste

{'corpus': 'corpus_stemming_sem_stopwords', 'model': 'SMV', 'vectorizer': 'CountVectorizer', 'normalizer': 'TrucatedSVD'}


,text,class
0,Sábado + folga + chuva = maravilha da vida 🥰😍☺...,1
1,O pôr do sol de minha cidade. Amo demais 🥰 #Fo...,1
2,Ariana Grande medalhista do Adauto Bezerra&amp...,1
3,Dom Pedro Laguna - Aquiraz😀\nhttps://t.co/hz78...,1
4,Nossa pré-candidatura para Vereadora de Fortal...,1
5,Seara Praia Hotel-Fortaleza😀\nhttps://t.co/ghk...,1
6,Bristol Guararapes Hotel-Fortaleza😀\nhttps://t...,1
7,Mais um #turbulenciaafesta concluído com suces...,1
8,"#FalaJogador | Fala aí, meu REIMARINHO! 🥰👑\n\n...",1
9,É hoje!!! #CaminhadaRosa em #Fortaleza 🙌🏼🙌🏼🙌🏼\...,1


In [ ]:
fortaleza = ['Sábado + folga + chuva = maravilha da vida 🥰😍☺️ #fortaleza', 'O pôr do sol de minha cidade. Amo demais 🥰 #Fortaleza #Ceara \nFoto via @Juliana_Mara https://t.co/Nqxdm6oXEq', 'Ariana Grande medalhista do Adauto Bezerra&amp;\nZendaya da escola da prefeitura&amp;\nMichael Cera comendo salgado caro do Ari&amp;\nAnya Taylor Joy tomando chuva na fila do RU.\nCidade da Luz 🥰 #Fortaleza #Ceará https://t.co/5MNwy8z6eK https://t.co/nz0k2dM0lB', 'Dom Pedro Laguna - Aquiraz😀\nhttps://t.co/hz78rqgAHW🏝️\n**\n#aquiraz #fortaleza #ceara #eusebio #caucaia #rebaixados #maracanau #messejana #maranguape #navesinsanas #horizonte #navessocadas #dubstyle #itaitinga #fixa #alturalivre #portodasdunas #fortalezaceara #bhfyp @plosmarcelo✴️ https://t.co/BMA6Qc8xM9', 'Nossa pré-candidatura para Vereadora de Fortaleza tá bonita demais! É o apoio de vocês que nos joga pra frente e chega força do país inteiro.\nOlhaê o nosso querido @MarceloFreixo dando o recado 🥰\n#Psol #Eleições2020 #Fortaleza https://t.co/bNYBe6RAww', 'Seara Praia Hotel-Fortaleza😀\nhttps://t.co/ghkG4nZHHp👍\n*\n#fortaleza #ceara #mp_traveldestination #brasil #saopaulo #fortalezaceara #mp_brazilok #cear #nordeste #mp_brazilok_fortaleza #recife #fortalezace #riodejaneiro #amor #salvador #love #caucaia #bahia #natal #praia #brazil💥 https://t.co/6KOvGzd6Y6', 'Bristol Guararapes Hotel-Fortaleza😀\nhttps://t.co/WDf6XJygpr👍\n**\n#fortaleza #ceara #mp_traveldestination #brasil #saopaulo #fortalezaceara #mp_brazilok #cear #nordeste #mp_brazilok_fortaleza #recife #fortalezace #riodejaneiro #salvador #caucaia #bahia #maracanau #natal #brazil💥 https://t.co/EnsdyaMJbC', 'Mais um #turbulenciaafesta concluído com sucesso!! 🥰 Muito obrigado a todos que estiveram com a gente até de manhã! 😍 Vocês são demais!! 👏🏼👏🏼\n📸 @NaraFassi \n#SeTemXandAviãoEuVou #Turbulência #Fortaleza https://t.co/KqlEVy66us', '#FalaJogador | Fala aí, meu REIMARINHO! 🥰👑\n\n#VamosFortaleza | #Fortaleza | #FortalezaEC https://t.co/iMaUeVLd5N', 'É hoje!!! #CaminhadaRosa em #Fortaleza 🙌🏼🙌🏼🙌🏼\n\n😀Estou muito feliz em ser padrinho este ano desta… https://t.co/cDoWBTdd3i', 'Sendo feliz com os meus copos maravilhosos @anygabrielly @SPlotnikovaBRA @NowUnitedBR @JohnnyRocketsBr @PepsiBr  \nComprei pelo Delivery... corre lá e peça o seu... eu já quero fazer a coleção toda.. são 16 copos 🥰❤️ @JohnnyRockets #iguatemi #fortaleza @Nowun1ted @TeamNowUnited https://t.co/2VLIkbYAs5', 'Vem aí o XIII Evangelizar Fortaleza em casa. Confira o recado do Padre Reginaldo Manzotti. 😀\n➡ Viva esse momento de fé aqui na TV Evangelizar. 📺\n\n#XIIIEvangelizar #EmCasa #Fortaleza #PadreReginaldoManzotti #TVEvangelizar https://t.co/mfpIgd0itk', 'Bom dia, Fortaleza!! Café e tapioca todo dia de manhã. 😀👍 #Fortaleza #Nordeste https://t.co/TLkn4Gw4no', 'Bom dia😀👙☀️💋 #praiadofuturo #fortaleza @ Praia do Futuro https://t.co/VH23ycgwnl', '#tchauRogerioceni e os demais #Vitinho #cesar #Gustavohenrique #Arão #DiegoRibas #René #LéoPereira  alguns mais que esqueci nessa hora de Raiva @Flamengo a e vale lembrar parabéns ao @CearaSC #Fortaleza #Fluminense e demais #Flamengo #Mengão #CRF 🔴⚫🔴⚫ 😠 https://t.co/KEPbALOeJF', '#timecbde | Na #livedasemana relembramos hoje a campanha invicta do Brasil 🇧🇷 no 1º Mundial ISF de Futsal que foi realizado em 2007, na cidade de #Fortaleza (CE). Na ocasião, nossa equipe foi composta por alunos do Colégio Magnum (MG), e por alunas do Colégio Evolutivo (CE). 😀 https://t.co/YcDXMo3U2o', 'Volta feliz d #fortaleza 😀#tempoforte… https://t.co/G02KBWYFHM', 'Gosta de praia, biquíni e mora em #fortaleza ? \nTe convido pra seguir minha loja on-line de Beachwear. Lá no app vizinho 🥰 \n\nhttps://t.co/FTrSD3Q6Qg https://t.co/Obehzyqrrn', '#Fortaleza não decepciona.🥰 https://t.co/Zf1FzTZKQ4', 'Novidade no RioMar Kennedy! Conheça toda a qualidade e sabor da Kopenhagen. Aprecie deliciosos sabores a cada mordida. 🍫☕\n\n📌 Fica localizado no Piso L1, ao lado da Turatti.😀⠀⠀⠀\n⠀⠀⠀⠀\n#RioMarKenendy #NovidadenoKennedy #Novidade #Fortaleza https://t.co/WcQb4i1zl0', 'Sim, eu sou cearense! 😀 #ceara #fortaleza #nordeste #terradosol @ Aterro da Praia de Iracema https://t.co/lDvRGqKbEJ', 'Já pode fazer checkin no Apagão 2015 - Eu Fui!? 😀 #Fortaleza', 'Bom dia meus amores 😎😀👙🌞😍💋 #fortaleza @ Marina Park Hotel https://t.co/C9P2npM5ah', 'Remando remando 😀☀️ rosteffler @anaaouriques #Gutinho #Fortaleza #CE http://t.co/ZNLUfGKsUV', 'Fim de Tarde com essa 🌕 maravilhosa 🥰\n#Fortaleza\n#Brazil https://t.co/04ydno30H7', 'Sensualize com estilo 🥰\n#esperamarido #lingeries #vendasonline #modaintima #vaidarcerto #centrofashion #moda #modaplus #Fortaleza #Ceará #picpay #fazumiti #MaisVoce https://t.co/e8s42vnBle', 'Chega junto! Porque vai ter circo na sua casa! \n\n🎪 Dia 3, 10h da manhá no canal 8.1 @tvcidadefortaleza \n\nSintonize a FM 89.9 #ARádioQueCombinaComVocê 😀\n\n#89FM #CombinaComVc #Fortaleza #Rádio89FM #Circo https://t.co/y3QDz4VtG4', 'Alô @Atletico e @GaloNaVeia no domingo não estarei no @Mineirao pela terceira vez (#FORTALEZA, #GREMIO E #AMERICA) Pois o seu parceiro comercial, @GaloBmg cancelou meu cartão, e não emite um novo, impossibilitando o processo de compra on line! Lamentável 😠😠😠', '#TBT Com a camisa do ídolo!! 🥰❤️ #MarceloBoeck #Fortaleza https://t.co/VCMtTMtQrD', 'A mochila a entrar na Fortaleza de Valença! 🎒 É uma das principais fortificações militares da Europa  e com cerca de 5 km de perímetro amuralhado. 😍 Obrigado pela foto Exmo! ❤️ #Fortaleza #Valença #Minho 😎 #Portugal 🥰 #ViagemàVista 🧳 #ViagemàVistaPeloMundo ✈️ https://t.co/HXSE3mysK1', 'Meu amor e meu xodó na mesma mesa 🥰 #Fluminense  #Fortaleza https://t.co/mpF3KUWo1K', 'Amo esse climinha no meu país! Os dedos dos pés geladinhos 🥰 #Fortaleza https://t.co/yav47somqT', 'Pros fãs de #musicais da #Disney em #Fortaleza :\no #CineSãoLuiz vai exibir #Moana em #3D esse fim de semana !\n #GRATUITO 😀\n@cidade_fortalez https://t.co/XRdkH2KyNN', 'Uma boa pra quem tá afim de conhecer o #rugby aqui em #fortaleza #ceará 😀 https://t.co/eNHcfd9k5U', 'Olha só recadinho da ex dançarinas do @AvioesdoForro_ @XandAviao @SolAlmeidaa 😀\n#fortaleza #aviõesdoforró #xand #solange #brasil #indico https://t.co/VO0Ik59e3j', 'Defina em uma palavra o ano de 2016 pra você? 😀\n\n#MarketingDigital #Ano2016 #brasil #fortaleza', '@tavasconcellos sabe o q mais me diverte pela manhã!? É ouvir vc ri das besteiras do @BoechatBandNews e do @jose_simao. #fã #Fortaleza-Ce 😀', '2016 e tem torcedor q ainda acha que foguete na madrugada ganha jogo. 😠\n#juventude #fortaleza #serieC https://t.co/FUpuYMWlpm', 'Deixando o primogênito no aeroporto para seguir de volta à nova casa, após os feriados conosco. A gente te ama, Vini! 🥰♿️🙌 #travel #Fortaleza #ceara em Aeroporto Internacional de Fortaleza https://t.co/W29UBMKhnU', 'O que tem hoje em #fortaleza pra um casal de férias? 🥰 https://t.co/SxPR1mEAUZ', 'Pelo o menos essa noite vou dormir feliz, obrigado #Fortaleza 🥰🙏', 'De volta 🥰😉 #boy #girls #brasil #hot #bissexual #leke #hot #fortaleza #ce https://t.co/9VmI5CVr3d', '“Moro num país tropical abençoado por Deus e bonito por natureza” 🎼\nE minha cidade é mais linda ainda 🥰\nFortaleza, capital do Ceará, nordeste do meu Brasil 🇧🇷\n.\n.\n.\n.\n#fortaleza #ceara #nordeste #iateclube #sabadou… https://t.co/K6LPH8Tq9R', 'Bom dia! 😀\nPor mais que tudo esteja difícil e que você se sinta sem esperança, não desista! \nDias melhores sempre chegam. \n\nQuem crer digita AMÉM. \n\n#RonaldoMartins #NãoDesista #DiasMelhores \n#Fortaleza #Ceará https://t.co/C0B4oERcLW', 'Um filme com fortes emoções será exibido no #CineRecordEspecial nesta terça-feira (21): Uma Noite de Crime. Filme inédito, na tela da @tvcidadefortaleza 📽\n\nSintonize a FM 89.9 #ARádioQueCombinaComVocê 😀\n\n#89FM #CombinaComVc #Fortaleza #Rádio89FM https://t.co/PknoMuYaBq', 'Hoje é dia daquele que tá sempre pronto para  ouvir, acolher e mediar. Feliz Dia Nacional do Ouvidor! 😀  \n \n#Ronaldo Martins #DiaNacionaldoOuvidor  #Ouvidor #Fortaleza #Ceará https://t.co/WSfHjwab4E', 'Hoje é o dia de quem trabalha para nos fazer sorrir! Por isso, quero parabenizar todos os comediantes, em especial para aqueles que eu já tive o prazer de trabalhar.  😀\n\n#RonaldoMartins #DiadoComediante #Comediante #Bolachinha #Fortaleza #Ceará https://t.co/yYfqCH2Rzn', '#Cearense2018 #FORTALEZA #CEARÁ #FORxCEA       \n\nQUEM É O MAIOR E CAMPEÃO DESSE BRASIL ⚽️😀👊🤞', 'Da série: Por onde andei... 😀 #PraiaDeIracema #Fortaleza #Ceará #AmoViajar #Viagens #Trip… https://t.co/2wrNPcMRNh', 'A cidade que dorme e ilumina a paisagem. Boa noite, Fortaleza, sua linda.🌃👍🏼👏🏼🙌🏼✌🏼😀😍😘🌃#fortaleza… https://t.co/ck97CgHQi1']

Morte = ['O #amor gera a #vida; o #egoísmo produz a #morte. Ninguém pode #serfeliz se não for amado; se não fizer uma experiência de amor. 🥰❤️ Leia o texto e compartilhe! #relacionamento #cancaonova \n\nhttps://t.co/LzFEXOuwMa', 'Vejam o que acontece quando se tenta registrar a #morte de alguém 24h depois da #pfizer ...\n\n😡 Óbvio\nMil e um truques p manter os números baixos... https://t.co/jKDtEyr1Uu', 'O #amor gera a #vida; o #egoísmo produz a #morte. Ninguém pode #serfeliz se não for amado; se não fizer uma experiência de amor. 🥰❤️ Leia o texto e compartilhe! #relacionamento #cancaonova \n\nhttps://t.co/LzFEXOuwMa', 'Um pedacinho De Ontem campeonato ESPORTE QUE EU AMO .. \n#EQUIPEKILLER #Ou #A #MORTE #Oss 👊🏻😤😠🖕🏻 https://t.co/ORn6XK6ptN', 'Um país onde uma pessoa que trama a #morte dos próprios pais, de maneira covarde e cruel, recebe #indulto de #DiaDasMaes.\n\nÉ #imoral. 😡', 'Sério tá feio já 😡realmente tá pagando mico 😳😤😵😱 #morte', 'SE A #PRODUÇÃO DESSA @fazendarecord NÃO COMEÇAR A EXPULSAR. VAI DAR #MORTE. TODOS ESTÃO TENSOS LÁ DENTRO! 💣🔪🔨👊🏽😡😤💩 https://t.co/2D9WkFSXAx', 'Toda tragédia é anunciada nesse buraco de q é esse desgoverno😡\n#BozotrazFomeeMorte\n#Morte\n#DaCultura\n#Ciência\n#Saúde\n☠💀☠💀☠💀☠💀 https://t.co/87clFPNFgc', 'io a cosa pensavo quando ho scelto di fare tedesco?!?! 😡 #morte 🔫', 'vontade de tirar uma sonequinha deliciosa agora mas tenho prova amanha 😠😠😠😠 #morte ao capitalismo', 'E se você que está batendo palma para todo esse circo que está sendo montado, você é cúmplice. Cê entendeu, né?!😠\n*\n*\n*\n#brasil #coronavírus #pandemia #lockdown #quarentena #injustiça #morte #paísesemdesenvolvimento #fome #crianças #mães https://t.co/c4rm91hVP6', 'ai🙄🙄 gente muito bom ✅✅ comer no bandejão 😡😡 da melhor ⭐⭐ do brasil💯💯 #faculdade #estudos #comida #morte #entp_de_magé', 'desmaiei no banho, bati a cabeça no chão e não para de sangrar😀\n#morte', '@SorakaAgressiva @_luiz_hansen_ Sorakinha, vc me respondeu 🥰\n#Morte aos jannistas', 'nossa radfem é hilario JKKKKK 😍🥰 VAMOS ESPALHAR O &gt;&gt;&gt;&gt;FEMINISMO&lt;&lt;&lt;&lt;&lt;&lt;&lt; (femismo) usando a paz e o amor! 😃 #morte ao penis (meme da gretchen) VAMOS SER TRANSFOBICAS E ABOLIR GENEROS UUUUUUUUUUU MORTE AO PENIS KKKKKKKKKKKKKKKKKK HILARIO 😍😍', 'Ordens são Ordens não é Eduardo Pazuello? Mesmo que isso acabe tendo como consequência a #morte de milhares de #pessoa 😡\n\n#forabolsonaro #brasil #politica #academiambl #movimentobrasillivre #eleição2022 #bolsonarogenocida #bolsonaro #ministerio #pandemia #health #saude https://t.co/14DIwcaJjU', '@Estadao @EstadaoPolitica Atitude #DESPREZÍVEL , especialmente diante da #morte de mais de 20.000 #brasileiros 😡', 'só pude pegar o cell pq eta intervalo d aula, meu pai vai pegar ele dnv agr\n#morte 😀🔫', 'Gente até quando vão acontecer mortes por causa de bandido que se infiltra nas torcidas. Isso me deixa arrasada. Tem que mudar essa Lei e torna-la mais dura. 😭 #Briga #Torcida #morte Isso não é torcedor é Marginal! 😡 #CidadeAlerta', '@GreenpeaceBR 👿😈#Ruralistas 😡#Agro ☠️#Agrotóxico 💀☠️#Câncer  \n🌶️🌶️NO ⚫️ DOS OUTROS É 😱😱REFRESCO🎃⚰️☠️#Morte', 'A #MORTE não é  a solução para os  seus #PROBLEMAS!!! \n✌😀✌\n\n#SetembroAmarelo💛\n#DigaNÃOaoSuicídio💛\n#MateEssaIdeia💛\n#ValeAPenaViver💛\n#AlegriaVerdadeira💛', '😀😀😀😀😀 regrann from @aalpino  -  Legado - Cartum para Playboy. Janeiro, 2016. #morte #familia… https://t.co/DlcSD3kWha', 'Qual seria seu último desejo antes de morrer?\nhttps://t.co/S0TBHc0QEn 😀\n#humor #quadrinhos #engraçado #morte https://t.co/T44SerqzL9', '#AGORA #NINGUÉM #VAI #ACREDITAR #NA #MORTE #DOS #URSOS, #MESMO #VENDO #ELES #MORREREM. \n😠😠😠😠😠😠😠😠😠😠😠😠😠😠\n ****** Os seres desumanos já chegaram, em muitos lugares... https://t.co/VHrUpVLfc2', '😠😠😠Não é #AMEAÇA e sim um Aviso Prévio , Mexa com a familia de um homen e tenha uma certeza a que a sua #Morte é... https://t.co/vz2gA2asyt', 'Não queira abraçar ela na minha frente #ciumes #morte 😉😠😬🔫🔪💣', 'Quem inventou horário de verão merece a #morte. 😬😠😬', '@itsmep3dro #morte pedro 🥰❤️', 'Eu &amp; ela, a #morte 😍🥰 https://t.co/IeAQv2FKrL', 'há. Agora ficô Bão 😂🙃🙃🙃😂\nCom isso, o #STF tem duas #Decisões #Conflitantes sobre a #Demanda.\nPor quem irão #Decidir?\nPela #Morte ou pela #Vida\nPelo(a) 🤥🤑😈👿😈🤑🤥 ( #Capiroto )\nou\nPelo(a) 🥰🇧🇷😇😇🇧🇷🥰 ( #Humanidade )\n\nhttps://t.co/ts5DFtohmX', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #telhados #casa #casas #house #pulo #like4likes #likememe #nature #videos #video #instavideo #videoviral #engraçado… https://t.co/ts4bDN6g3B', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #lenhador #arvore #madeira #tree #machado #like4likes #likememe #nature #videos #video #instavideo #videoviral #engraçado… https://t.co/4MqEjfsqO8', 'Mais um #capítulo\nda #série #emorreu\nHoje tem dois heim.\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #adestrador #jacarei #zoo #zoologico #alligator #like4likes #likememe #zoologia #video #videos… https://t.co/2J0xfoB19A', 'Mais um #capítulo\nda #série #emorreu\nHoje tem dois heim.\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #bicicleta #bike #fogo #fire #welcometoibiza #caixao #dança #seriados #videos #video #videogram… https://t.co/WKPVUO9i0P', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #garrafa #bottle #bola #ball #welcometoibiza #caixao #dança #seriados #videos #video #videogram #instavideo #videofun… https://t.co/CqNFmaODe6', 'Mais um #capítulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #dead #luz #redeeletrica #blackout #cemig #eletricidade #eletricista #curtocirtuito #welcometoibiza #caixao #dança #seriados… https://t.co/I7pO2lTAPn', 'Mais um #capitulo\nda #série #emorreu\n*\nOuça a gente vai. Informações na Bio 🎧🥰\n* \n#morte #mortefeliz #dead #paraquedas #aviao #saltodeparaquedas #caiudeparaquedas #aeronaves #welcometoibiza #caixao #dança #seriados… https://t.co/7I5BtS3cxl', 'Faltou ontem porque todos estavam em #lives de #shows no #youtube mas hoje tem dois capítulos de #emorreu \n*\nOuça a gente vai. Informações na Bio 🎧🥰\n*\n #morte #dead #🔞 #welcometoibiza #caixao #mortefeliz #dança… https://t.co/egX8iXVovL', 'Leitura do dia 🥰 Livro Viver o seu Morrer de Stanley Keleman.\n\n#livros #luto #morte #viver #psi #psicologo #psicologia #psicoterapia #saudemental #bemestar #psicologiaporamor #procureumpsicologo… https://t.co/HIk6p0Q23m', 'NUNCA ESQUEÇA: #lula do #pt APÓIA A russia NESTA #GUERRA E CONDENA A #UCRÂNIA, COMO SE A CULPA POR TODA ESTA #DESTRUIÇÃO E #MORTE DE CIDADÃOS FOSSE AGORA DO PAÍS QUE ESTÁ SENDO DEVASTADO!!\n\nVote sim no #exPresidiário Brasil e verás o que lhe espera!!! 😡 #povoBURRO https://t.co/2YbgDINlz1', '😡 COVARDIA\nA mãe confessou ter notado um sangramento após o homem ficar a sós com a neném.\n\n🔸️ Saiba mais sobre o caso ⤵️\nhttps://t.co/C4ncbJpF2M\n\n#prisão #covardia #santacatarina #estupro #morte #nenem #pcsc', 'Esta é a verdadeira #democracia da #esquerda: #guerra, #medo e #MORTE!! 😡 Prepare-se Brasil: #lula do #pt está voltando! https://t.co/T0WmwnsXek', '3 dias com dor de cabeça da sinusite.\nEitaaaa dorzinha infernal!\n😡😡🤬🤬🤬\n#sinusite #morte #aaaaaaaaa https://t.co/y0UvHfLXtp', 'Sabemos onde está a #mulher que sofreu #assalto na #Supervia, só não sabemos do assaltante 😡\n@PCERJ @PM_RJ @SuperVia_trens #assalto #roubo #morte #violencia @radiotupi  @cbnrio @bandnewsfmrio', '@jurandirsas Saudações Patriotas !🇧🇷\nEu gostaria de poder ser à #Morte. Por um só dia, para poder mandar para as profundezas do infinito, todos esses... \nAntipatriotas, Corruptos, Comunistas, Esquerdistas, Neo Nazistas, #PTistas, #FabianosMarxistas, Terroristas, E criminosos de modo geral.😡 https://t.co/YO71jxmcgZ', 'Em Contagem (MG), 50 cães morrem com suspeita de envenenamento. Animais estavam no Lar Temporário, localizado no Bairro Quintas do Jacuba 😡https://t.co/pGXxZ4b1Su #cãesegatos #abrigo #animais #morte #investigação #envenenamento #denúncia https://t.co/1JuDdcY2ql', '@aureoboy sim! isso é pouco, mi amigo! o sabonete infelizmente deixa rastros e bacterias espalhados em todo meu corpo! um absurdo.... tem que morrer mesmo #morte ao sabonete #peste 😡', '@Pontifex_it @Avvenire_Nei @oss_romano @iamCARITAS #Voce grossa contro chi #produce #morte!!!😡 https://t.co/nS6KWURiNh', '@clagraca #VivaVenezuela ✊😡#VivaLaRevulucion #Morte À @jguaido Puto Desgraçado', '@Calks_Real seu machista homofibico  😘😘 eu sou a vitoria machistona do pedaço adoro homem e vc vai ter que se submeter a mim 😡🤬🤬\n#morte ao penis KKKKKKKK 🥺🤣😂😂😅😆']

Acidente = ['Deu ruim😡😢#plantaoenfoco\nAcidente com ônibus deixa feridos na Alameda\nLeia a matéria: https://t.co/JUdHA8Xrki \n\n#niteroi #transito #acidente #plantaoenfoco https://t.co/MvaAX3hENp', 'Bom dia para o “igenheiro” q coloca a segurança em 1 lugar! 🙄😒😡\n#segurancadotrabalho #transito #estrada #acidente https://t.co/0Q1li8ruhB', 'Enquanto isso... pelo salto Merece uma nota 8... encostou a mão no chão.... 🤣😂😅😆😁😃😀😀😃😄😆😅😂🤣🤣\n\ndiga ai qual nota você daria ?\n\n#Salto #acidente #olimpíadas #saltoornamental em Parauapebas https://t.co/yv73qTTjff', 'Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguro #seguroauto #seguromotocicleta https://t.co/fXUsosMfrE', '🚗Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguro #seguros #seguroauto #automoveis #auto #carro #protecao https://t.co/fsfCcvopSt', 'Nunca sabemos quando um #acidente pode acontecer. Por isso, estar #prevenido nunca é demais. 😀 Nós trabalhamos com diversas opções de seguro. Assim você encontra aquela que mais tem a ver com sua necessidade.\n\n#seguroviagem #viagem #seguro #planodesaude #planosdesaude https://t.co/PgVSnU9wQz', '#gt #bikegt #bicicleta #bicicletagt #acidente #atropelado \n.\n😶🙃😶😀😐🙃😀🚲💥🚗 \n..\nTa bom.... Foi… https://t.co/3iUWwo0AUp', '😱😱😱\nE ainda fugiu sem prestar socorro!\nInadmissível o tamanho da audácia 😠😡\n\n.\n\n#prontofalei #peoples #PARE #acidentes #duasrodas #atropelamento #perigo #sqn #zoando #pessoa #rindo #não é o que #parece #acidente… https://t.co/iy2zBxqsXF', '#Acidente após entrada da Ilha RT @DanyeleDanyreis @InformeRJO Av. Brasil parada na seletiva sentido zona Oeste pq? 😠', 'Que explicação tão torpe 😡😡😡\n\nEntão a vítima foi a viatura? A vida perdida foi a da viatura? #apanhadosrtp #portugal #apanhados #rtp #eduardocabrita #acidente https://t.co/t2LaCf6sEU', '😱😱😱\nE ainda fugiu sem prestar socorro!\nInadmissível o tamanho da audácia 😠😡\n\n.\n\n#prontofalei #peoples #PARE #acidentes #duasrodas #atropelamento #perigo #sqn #zoando #pessoa #rindo #não é o que #parece #acidente… https://t.co/iy2zBxqsXF', 'Acidente na #rodoviacastelobranco tudo parado sentido interior. 😡#acidente #castelobranco #rodovia #transito', 'Mais é um carai mesmo 😡😡😡 #transito #acidente #fernaodias #mioorolê @ Guarulhos S.p https://t.co/tCbdKflxGL', 'Meu dia hoje começou de primeira! 😩😡🚌🚎🚐🚑🚒🚓🚗🚕🚙🚚🚛 #Engarrafamento #TudoParado #EPTG #Acidente #Taguatinga http://t.co/8kBugXhH86', 'Eu desisto! Toda vez é isso!!!! 😒😒😒😡😡😡😡 #engarrafamento #acidente #estrada #partiuBE #stresse #BR101 @… http://t.co/Ggdy5IW6qc']

Marvel = ['Esto me da vida 🥰 #Marvel #MightyThor #GirlPower https://t.co/m6wmSHGNu7', 'Metadinha #TChalla e #Nebulosa! 😍🥰\n\nQuem aí shippou o casal de #WhatIfMarvel? \n\n.#WhatIf #Marvel https://t.co/CeCST5kOF1', 'BABY YODA DE LOKI PASSANDO EM SUA TELA! 😍\n\nQUANTA FOFURA! 🥰\n\n#BabyYoda #StarWars #Loki #Marvel https://t.co/xsW2kHNcfk', 'QUE FOFURA! 🥰\n\nE esse cosplay de #MoonKnight! Ficou perfeito! 😍🌙\n\n#Marvel #CavaleiroDaLua https://t.co/576pIB1xoS', '“Como você prepara sua massa?” DO JEITO QUE VOCÊ QUISER, @dannyramirez 🥰 A @disneyplusbr ouviu o pedido das migas e trouxe o próprio Neném do Macarrão pra bater um papinho com a gente! Foi um sonho, olha só: https://t.co/3MVdPmTcQS #TFATWS\xa0#Marvel #EntreMigas https://t.co/2XRTofrh11', 'Aproveitar que essa é a última semana de felicidade no lokiverso e postar meu desenho do #kidloki \nRt + fav = 🥰\n\n#arte #desenho #art #Loki #LokiWednesdays #lokialligator #Marvel https://t.co/4GdUA8NY9E', 'Eu tô paralisada com esse treiler de moon knight nova série da Marvel Studio 😮🥰\n\n#MoonKnight #Marvel https://t.co/VxYZkxgUwa', 'QUEREMOS 🥰\n\nKaren Gillan tem interesse em fazer uma série como Nebulosa no Disney Plus.\n\n“Eu acho que ela tem uma história muito interessante, então seria muito legal saber mais sobre." #Marvel #GuardiansOfTheGalaxy https://t.co/p6GaSOtQYb', 'WANDAVISION❤️(GIF)\nQuem já assistiu? Tem sido uma das minhas séries favoritas ultimamente 🥰\n•\n\n#wandavision #wandamaximoff #wanda #wandafanart #marvelbrasil #marvel #disney #disneyplus #seriedisney #procreate #art #arte #illustration #ipadart #digitalart #fanart #desenho l https://t.co/XmxKD7YVyp', 'Quando o me perguntam porque amo tanto a Marvel 🥰😍❤🤜🏻🤛🏻\n#Wiccano #Hulkling #MarvelStudios #Marvel https://t.co/BCEa7EnARQ', 'Meu desenho da Sue Storm-Richards, a Mulher Invisível do Quarteto Fantástico. 👱\u200d♀️4️⃣\n\nJá estava devendo a um bom tempo, se você puderem curtir e da RT e muito importante 🥰\n\nNão deixem flopar 🥺\n#Marvel #Fanarts #FantasticFour #InvisibleWoman #SueStorm #QuartetoFantastico https://t.co/s8qhB4sm1T', 'E a performance cômica do ano vai para:\n\nWanda e Visão 🥰\n\nEsse casal começou com o pé direito! \n\n#WandaVision #Marvel #DisneyPlus https://t.co/biYAp4Bccn', '🐕\u200d🦺 Bienvenue à #Marvel (2 mois), nouvelle recrue de la famille ! 🥰 https://t.co/zzpwuB24SU', 'Namoral amei essa aquarela ❤️\n\n🔄+❤️= Eu felizinho 🥰\n\n#fanart #Deadpool #Marvel https://t.co/Fdo8DYpxG2', 'Qual deles é o melhor HULK na sua opinião? Não esqueça de deixar seu voto nos comentários! 😠😠😤\n⠀⠀⠀⠀⠀⠀⠀⠀⠀\n#hulk #escolha #melhor #maisgeektv #filme #mcu #marvel #cinema https://t.co/wawiTJPsGl', 'HAPPY BIRTHDAY @Fer26Ari ❤❤\n\nHoje nossa administradora está completando 18 anos 🥰\nParabéns Fernanda (Nossa Capitã Marvel), aproveite seu dia! A Legião te ama ❤\n\n#WandaVision #Marvel #CaptainMarvel https://t.co/3nMHAQXd5C', 'deadpool pq sla só deu vontade d fazer nem curto tanto marvel.\n.\n.\n.\n.\n.\n( Plz Fav❤️&amp; RT🔁= i happy 🥰)\n.\n.\n.\n.\n.\n(#Art #drawing #anime #animeArt   #originalcharacters #fanart #doodles #sketch #digitalart #MyArt #ArtistOnTwitter #Deadpool #marvel) https://t.co/P96GOii8jP', 'Coleção atualizada!\n#nerd #HQs\n#DCComics #Marvel #DragonBall #StarWars\n#IronStudios #Eaglemoss #Kotobukiya\n#Bandai\n#Superman #Thor #Shazam #Batman\n#WonderWoman #HalJordan #Thanos #Galactus #Darkseid\n✌✌✌✌✌\n😀😀😀😀😀\n👊👊👊👊👊 https://t.co/orNma7ZknD', 'Rostinho novo para a página ✨❤️\n\nps: tô muito feliz com o resultado desse desenho 🥰\n\n#JeanGrey #XMen #Marvel https://t.co/SBrE6QQtZQ', 'Os Defensores (@TheDefenders) na bela arte de Neal Adams. 😀\n#JessicaJones #IronFist #PunhosdeFerro #LukeCage #DareDevil #Demolidor #Marvel #Netflix #f2fstore https://t.co/PzA6Oy6GYI', 'Vou postar aqui pq o insta ferrou cm bagulho, fiz uma arte do aranha que me orgulhei. Finalmente 🥰\n-\n-\n-\n-\n#art #digitalpainting #draw #drawing #artwork #marvel #spiderman #ilustra https://t.co/Qf2QugjQ2Q', 'Te amo #marvel 🥰 https://t.co/RMgMZnbypb', 'Não priemos cânico!" 😀🙃😛\nAdorei esse Chapolin do amigo @EricBlakeArt ! Tive que colorir!\nCores por mim Marcos Martins @mmartinsart .\n#art #chapolin #chapolincolorado #avengers #infinitywar #vingadores #marvel #televisa #thor #thanos #artist #illustration #sketch #mmartinsart https://t.co/e6qjrGgPmP', 'Um sonho que se realiza.\n\nCosplay do Wiccano/Demiurgo da Marvel. 🥰\n\nAgora é iniciar as gravações do meu projeto, e depois esperar pela CCXP 22 para usar ele pleno.\n\n#BillyKaplan #Wiccan #Marvel #MarvelCosplay #WiccanCosplay #Demiurge #Cosplay #YoungAvengers #JovensVingadores https://t.co/3X18LncioY', 'Já estamos com saudade dessa variante.💚 (E de todas as outras)\n\nSylvie, a gente te ama! 🥰\nSophia Di Martino também te amamos!\n\n#Loki #Marvel #Disney #Sylvie #Mobius https://t.co/LPSuN9oHHY', 'Alguns cadernos à pronta entrega com envio imediato! 🥰❤😍\n#OneDirection \n#StarWars \n#TheMandalorian \n#SHINee \n#Marvel\n#Given https://t.co/XAoLYabGXC', "#JTComics #104 : Quand t'assumes pas tes dessins chez #Marvel, un #comics GRATUIT par le papa de #Deadpool...😀 https://t.co/h9gbIeIohK", "Oii gente, Quinto desenho finalizado :)\nThor, Deus do Trovão\nComissões abertas de desenho\nChama DM pra orçamento hehe\n[Rt's são bem vindos :)🥰]\n #SixFanarts #ilustração #Marvel #Thor #thorloveandthunder https://t.co/R5qM8pso4N", 'reizinhos 🥰\n\n#NewMutants #IllyanaRasputin #Magik #RobertodaCosta #Sunspot #fanart #Marvel #XMen https://t.co/vKjJqziJK6', 'Conhece a Mulher-Hulk? 😀\n\n#Marvel #marvelcomics #MarvelStudios #Disney #DisneyPlus #TV #serie #culture #cultura #news #noticias\n\nhttps://t.co/JYoXR41JZo', 'Opaaa já estamos na Nerd Party.  😀\n#manaus #evento #venham #Geek #marvel #gotg #dccomics #batman #wonderwoman #aquaman #flash #greenlantern #nerd https://t.co/fHetWNxRZ2', '😠\n\nSiga @AlternativaNerd\n\nTAGS\n#alternativanerd #mundonerd #thor #marvel #games #gamers #hqs #mundogamer https://t.co/cJ6G80iURp', 'Mano eu tô super ansioso por esse Thor 4 💥\n\nDepois dos críticos só elogiarem o filme então. 🥰\n\n#ThorLoveAndThunder \n#Marvel', 'Quem disse que  #Marvel e #Broadway não combina? O Hugh Jackman está em cartaz com The Music Man e um fã resolveu fazer graça durante o agradecimento! 🥰\n\nObs: Queria deixar claro q essa adm apoia o musical dos Vigadores mostrado durante a série do Gavião Arqueiro... 👀 https://t.co/Ct1QM47acf', '❤ FELIZ ANIVERSÁRIO Chris Evans❣❣Obrigada por nos presentear com o Capitão América. 💞Toda a felicidade do mundo em sua vida.🥰 #Marvel @szilviade_ @Arcelin31705516 @ChrisEvans https://t.co/iiL1BR6Pid https://t.co/I1hdRHAYIy', 'Grupo: MARVEL.  \n\nCASO TENHA INTERESSE EM ENTRAR MANDO O LINK NA SUA DM. 🥰😘 \nCERTIFIQUE-SE QUE A SUA DM ESTÁ LIBERADA OU ME SIGA PARA MANDAR. 👍🏻\n\nRT PARA DIVULGAR. 🤗 \n \n#marvel #MarvelStudios #MarvelsAvengers #MarvelComics #hq https://t.co/lg9T6xk9i8', 'Bloopers - Blindspot S03xE13\nEu amo erros de gravação e AMO MAIS ainda a Risada da JAIMIE 😍❤🥰\n.\n.\n#jaimiealexander #blindspot #pontocego #janedoe #remibriggs #ladysif #thor #thor4 #sullivanstapleton #audreysparza #ashleyjohnson #ennisesmer #mcu #nbc #marvel #disney https://t.co/R5AzJTYPB0', 'Já que é aniversário da #ScarlettJohansson 🥰 fica aqui nossa singela homenagem e a prova que a #Marvel demorou pra fazer o filme! ➡️ https://t.co/7nmCYelqDr', 'Eaiii galerinha do twitter!?! Já fez sua boa ação de hoje??? Que tal doar um chocolatinho pra uma criança só pra fazer a Páscoa dela mais feliz?? 🥰🐰🍫 chamaaaa 🗣❤️\n.\n.\n.\n.\n.\n.\n.\n.\n.\n#pascoa #pascoa2022 #MarvelStudios #JustinBieber #SpiderMan #DoctorStrange #TomHolland #Marvel https://t.co/PmzfnwOdvL', 'É lindo demais ter uma amiga como a Maria. Ela acertou em cheio no amigo oculto. Obrigado, pequenina! 🥰🙌🏻 #Marvel #SpiderMan https://t.co/mlcyftZcTS', 'A primeira série de animação da #Marvel e #DisneyPlus chega a 11 de agosto! 🥰 #WhatIf\nhttps://t.co/6U2IpONqTJ', 'É pedir de mais pelo menos a animação? 🥰😂\n\n#homemaranha #tobeymaguire #marvel #universomarvel #tomholland https://t.co/UuKr55cMvW', '❣Um lindo dia aos apaixonados ❣#Marvel 🥰 https://t.co/H7LsV2z2hl', 'A escola em que minhas filhas estudam, hoje realizaram a entrega de um kit estude em casa. Com isso o aluno poderia escolher dois livros de presente. Olha o livro que ela escolheu e disse que é presente pra mim. \n\n#BlackPanther #Marvel 🥰\n\nO #SherlockHolmes é dela! https://t.co/zDLuDaV2U5', 'O Wolverine chegou no @fortnite galera! 🙌🙌🙌\nE hoje vamos fazer uma  live ESPECIAL as 22h na @twitch para tentar liberar essa Skin top! 👊😀🎮\nLink https://t.co/Tr84CJuswE\n\n#wolverinefortnite #fortnite #twitch #twitch #angryplaygamers #epicgames #marvel #wolverine https://t.co/sAfe8vn0tg', 'Adorei o filme Homem-Aranha no Aranhaverso! Bem divertido, bem feito, emocionante. \n\nJá tô atrás da trilha sonora 😀\n\n#HomemAranhaNoAranhaverso \n#Marvel', 'O sorriso mais lindo 😀❤✌\n.\n#finnjones #ironfistseason2 #ironfist #punhodeferro #dannyrand #thedefendersnetflix #thedefenders #lukecageseason2 #netflix #marvel #marvelcomics #marvelstudios #marvelironfist #marvelcinematicuniverse #marvelnetflix #heros… https://t.co/DucjVViWHV https://t.co/JCXeJXztqy', 'Capitão América vs Próxima Meia-Noite \nCapitão pegou a referência lançada pela Próxima com sucesso!\nMarvel Libera logo esse trailer pleaseee!\n#AvengersInfinityWar #Avengers4 #Marvel #Avengers ☺️😀🤓🧐📽️💻 https://t.co/iqwFvzUI4N', 'Bom agora que eu vi #Ultimato ou #AndGame nunca mais assisto filmes #Marvel que porra fizeram com a Romanoff e o Rogers?? Mano tava tudo bom ai cagaram tudo pqp. #TomanocuRoteristademerda 😠😠😠😠😠😠😠😠😠😠😠😠😡😡😡😡😡😡😡😡😡😡😡', 'É hoje papai… as 19hrs 🥰\n#Marvel #doutorestranho2 #DoctorStrange2 #MarvelStudios #DrStrangeMultiverseOfMadness #DoctorStrange # https://t.co/5JVmbnzIBm']

In [ ]:
# brasil[:5]


In [ ]:
teste_morte = []
teste2_acidente = []
teste3_marvel = []


for tweet in Morte:
    teste_morte.append(lematization(remover_stop_words(formatar_texto(tweet))))

for tweet in Acidente:
    teste2_acidente.append(lematization(remover_stop_words(formatar_texto(tweet))))

for tweet in Marvel:
    teste3_marvel.append(lematization(remover_stop_words(formatar_texto(tweet))))


In [ ]:
pred_morte = tuned_pipeline.predict(teste_morte)
pred_acidente = tuned_pipeline.predict(teste2_acidente)
pred_marvel =  tuned_pipeline.predict(teste3_marvel)

df_teste = pd.DataFrame(zip(Morte, pred_morte), columns=["text", "class"])
df_teste1 = pd.DataFrame(zip(Acidente, pred_acidente), columns=["text", "class"])
df_teste2 = pd.DataFrame(zip(Marvel, pred_marvel), columns=["text", "class"])

df_teste.to_csv("../result_pipeline_test/morte.csv", index=False)
df_teste1.to_csv("../result_pipeline_test/acidente.csv", index=False)
df_teste2.to_csv("../result_pipeline_test/marvel.csv", index=False)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    df.tweet_text, df.sentiment, test_size=0.2, random_state=42
)

corpus_train2 = x_train.to_list()
labels_train2 = y_train.replace({"Positivo": 1, "Negativo": 0}).to_list()

corpus_test2 = x_test.to_list()
labels_test2 = y_test.replace({"Positivo": 1, "Negativo": 0}).to_list()


all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].fit(
    corpus_train2, labels_train2
)

print("______" * 30)
print(all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].best_params_)


In [ ]:
y_hat = all_scores["SMV__TfidfVectorizer__PCA__Scaler"]["tuned_pipeline"].predict(
    corpus_test2
)

y_hat


In [ ]:
accuracy_score(labels_test2, y_hat)


In [ ]:
f1_score(labels_test2, y_hat)
